### 开始

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import os

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import keras_tuner
from tensorflow.keras import backend as K

physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_visible_devices(devices=physical_devices[0], device_type='GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

In [ ]:
keras.backend.set_image_data_format('channels_last')

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [ ]:
#自定义matries 计算r2
def R2(y_true, y_pred):
    sst = K.sum(K.square(y_true - K.mean(y_true)))
    ssr = K.sum(K.square(y_pred - y_true))
    R2 = 1 - ssr/sst
    return R2

In [15]:
training_mode = 'model' # parameter
max_trials = 50

####          Nodes = [[32,16,8,4],[64,32,16,8],[128,64,32,16],[256,128,64,32],[64,32,16,8,4],[128,64,32,16,8],[128,96,64,32,16,8],[256,128,64,32,16]] 

#### Nodes_choice = hp.Choice('Nodes_choice',values = [i for i in range(len(Nodes))])

#### Model

In [ ]:
# Create the keras tuner model.
class MyHyperModel(keras_tuner.HyperModel):
    
    def build(self, hp):
        Nodes = [[32,16,8,4],[64,32,16,8],[128,64,32,16],[256,128,64,32],
                 [64,32,16,8,4],[128,64,32,16,8],[128,96,64,32,16,8],[256,128,64,32,16]]
        Nodes_choice = hp.Choice('Nodes_choice',values = [i for i in range(len(Nodes))])
        reg = None #keras.regularizers.l1_l2(0.) if hp.Boolean("regularize") else None
        LayerNodes = Nodes[Nodes_choice]
        encode_results = dict()

        input_layer = layers.Input(shape = (19,1))
        x = layers.Flatten()(input_layer)
        out = layers.Activation(activation=hp.Choice("activation", ["relu", "tanh"]))(x)
        out = layers.BatchNormalization(momentum=0.8)(out)
        encode_results['raw'] = out
        # encode part
        for i in range(len(LayerNodes)):
            out = layers.Dense(units=LayerNodes[i],kernel_regularizer = reg,name = 'encoder_'+str(LayerNodes[i]))(out)
            out = layers.Activation(activation=hp.Choice("activation", ["relu", "tanh"]))(out)
            out = layers.BatchNormalization(momentum=0.8)(out)
            encode_results[LayerNodes[i]] = out
        
        # decode part
        for i in range(-2,-len(LayerNodes)-1,-1):
            out = layers.Dense(units=LayerNodes[i],kernel_regularizer = reg,name = 'decoder_'+str(LayerNodes[i]))(out)
            out = layers.Activation(activation=hp.Choice("activation", ["relu", "tanh"]))(out)
            out = layers.BatchNormalization(momentum=0.8)(out)
            out = layers.add([out,encode_results[LayerNodes[i]]])
            #out = layers.Activation(activation=hp.Choice("activation", ["relu", "tanh"]))(out)
            #out = layers.BatchNormalization(momentum=0.8)(out)

        # last decode
        out = layers.Dense(units=19,kernel_regularizer = reg,name = 'last_decoder')(out)
        out = layers.Activation(activation=hp.Choice("activation", ["relu", "tanh"]))(out)
        out = layers.BatchNormalization(momentum=0.8)(out)
        out = layers.add([out,encode_results['raw']])

        #final layer
        final_output = layers.Dense(units=1,activation=hp.Choice('activate_dense_last',values = ['linear','sigmoid']),kernel_regularizer = reg,name = 'final_dense')(out)
        model = keras.Model(inputs = input_layer,outputs = final_output)
        learning_rate = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")
        model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate),loss='mse',metrics=[R2,'mae'])
        return model

In [ ]:
def build(LayerNodes):
    encode_results = dict()

    input_layer = layers.Input(shape = (19,1))
    x = layers.Flatten()(input_layer)
    out = layers.Activation(activation="relu")(x)
    out = layers.BatchNormalization(momentum=0.8)(out)
    encode_results['raw'] = out
        # encode part
    for i in range(len(LayerNodes)):
        out = layers.Dense(units=LayerNodes[i],name = 'encoder_'+str(LayerNodes[i]))(out)
        out = layers.Activation(activation="relu")(out)
        out = layers.BatchNormalization(momentum=0.8)(out)
        encode_results[LayerNodes[i]] = out
        
        # decode part
    for i in range(-2,-len(LayerNodes)-1,-1):
        out = layers.Dense(units=LayerNodes[i],name = 'decoder_'+str(LayerNodes[i]))(out)
        out = layers.Activation(activation="relu")(out)
        out = layers.BatchNormalization(momentum=0.8)(out)
        out = layers.add([out,encode_results[LayerNodes[i]]])
            #out = layers.Activation(activation=hp.Choice("activation", ["relu", "tanh"]))(out)
            #out = layers.BatchNormalization(momentum=0.8)(out)

        # last decode
    out = layers.Dense(units=19,name = 'last_decoder')(out)
    out = layers.Activation(activation="relu")(out)
    out = layers.BatchNormalization(momentum=0.8)(out)
    out = layers.add([out,encode_results['raw']])

        #final layer
    final_output = layers.Dense(units=1,activation='linear',name = 'final_dense')(out)
    model = keras.Model(inputs = input_layer,outputs = final_output)
    return model

In [ ]:
model = build(LayerNodes = [128,64,32,16])

In [ ]:
model.summary()

### 2001

In [ ]:
from sklearn.preprocessing import StandardScaler
year = 2001
data_train = pd.read_csv(os.path.join('D:/SGYL/SM_results_data/Train/split/','train_data_'+str(year)+'.csv'))
data_validate = pd.read_csv(os.path.join('D:/SGYL/SM_results_data/Train/split/','validate_data_'+str(year)+'.csv'))
data_test = pd.read_csv(os.path.join('D:/SGYL/SM_results_data/Train/split/','test_data_'+str(year)+'.csv'))

X_train = data_train.drop(['SM'],axis = 1)
y_train = data_train['SM'].copy()   
X_validate = data_validate.drop(['SM'],axis = 1)
y_validate = data_validate['SM'].copy()
X_test = data_test.drop(['SM'],axis = 1)
y_test = data_test['SM'].copy()

standarder = StandardScaler()
X_train = standarder.fit_transform(X_train)
X_validate = standarder.transform(X_validate)
X_test = standarder.transform(X_test)

X_train = X_train.reshape(X_train.shape[0],X_train.shape[1],1)
X_validate = X_validate.reshape(X_validate.shape[0],X_validate.shape[1],1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1],1)

print('X_train shape:',X_train.shape)
print('X_validate shape:',X_validate.shape)

In [9]:

def scheduler(epoch,lr):
    # 每隔30个epoch，学习率减小为原来的1/10
    if epoch % 30 == 0 and epoch != 0:
        print("lr changed to {}".format(lr * 0.1))
        return lr * 0.1
    else :
        return lr

callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=20,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
hypermodel = MyHyperModel()
tuner = keras_tuner.BayesianOptimization(
            hypermodel,
            objective=keras_tuner.Objective('val_R2', direction="max"),
            num_initial_points=50,
            max_trials=max_trials,
            overwrite = True,
            directory='D:/SGYL/SM_results_data/Bayesian_Opt/Full_ResNet/',
            project_name=('Full_ResNet_'+str(year)))
tuner.search(X_train,y_train,epochs=100,batch_size = 4096,validation_data=(X_validate,y_validate),shuffle = True,callbacks = [reduce_lr,callback_early_stopping],verbose = 2)

Trial 50 Complete [00h 02m 04s]
val_R2: 0.7861226797103882

Best val_R2 So Far: 0.8351989984512329
Total elapsed time: 01h 36m 23s


INFO:tensorflow:Oracle triggered exit


In [13]:
filepath="Full_ResNet_"+str(year)+"_best.hdf5"
callback_checkpoints = keras.callbacks.ModelCheckpoint(os.path.join('D:/SGYL/SM_results_data/check_points/Full_ResNet/',filepath),monitor='val_loss',save_best_only=True,verbose=1)
callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=50,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
callbacks = [reduce_lr,callback_early_stopping,callback_checkpoints]

if training_mode == 'model':
    model_best = tuner.get_best_models()[0]
else :
    model_best = tuner.hypermodel.build(tuner.get_best_hyperparameters()[0])
model_best.fit(X_train,y_train,epochs=300,batch_size = 4096,validation_data=(X_validate,y_validate),shuffle = True,callbacks = callbacks,verbose = 2)

Epoch 1/300

Epoch 00001: val_loss improved from inf to 0.00123, saving model to D:/SGYL/SM_results_data/check_points/Full_ResNet\Full_ResNet_2001_best.hdf5
105/105 - 2s - loss: 0.0011 - R2: 0.8450 - mae: 0.0252 - val_loss: 0.0012 - val_R2: 0.8350 - val_mae: 0.0261
Epoch 2/300

Epoch 00002: val_loss did not improve from 0.00123
105/105 - 1s - loss: 0.0011 - R2: 0.8448 - mae: 0.0252 - val_loss: 0.0012 - val_R2: 0.8328 - val_mae: 0.0262
Epoch 3/300

Epoch 00003: val_loss did not improve from 0.00123
105/105 - 1s - loss: 0.0011 - R2: 0.8451 - mae: 0.0252 - val_loss: 0.0012 - val_R2: 0.8342 - val_mae: 0.0261
Epoch 4/300

Epoch 00004: val_loss did not improve from 0.00123
105/105 - 1s - loss: 0.0011 - R2: 0.8446 - mae: 0.0252 - val_loss: 0.0013 - val_R2: 0.8315 - val_mae: 0.0266
Epoch 5/300

Epoch 00005: val_loss improved from 0.00123 to 0.00123, saving model to D:/SGYL/SM_results_data/check_points/Full_ResNet\Full_ResNet_2001_best.hdf5
105/105 - 1s - loss: 0.0011 - R2: 0.8455 - mae: 0.0252

In [14]:
#save model
save_path = os.path.join('D:/SGYL/SM_results_data/model/Full_ResNet/','Full_ResNet_'+str(year)+'.hdf5')
model_best.save(save_path)
if (os.path.exists(save_path)):
    print('save model for year:',year)

#save model config
import json
bestConfig=tuner.get_best_hyperparameters()[0].get_config()
f = open(os.path.join('D:/SGYL/SM_results_data/Bayesian_Opt/Full_ResNet/Best_Config/','Full_ResNet_'+str(year)+'.json'), 'w')
json.dump(bestConfig,f)
f.close()
print(bestConfig['values'])

model_best.evaluate(X_test,y_test)

save model for year: 2001
{'Nodes_choice': 7, 'activation': 'tanh', 'activate_dense_last': 'sigmoid', 'lr': 0.0038017118026419682}
1712/1712 [==============================] - 4s 3ms/step - loss: 0.0012 - R2: 0.8206 - mae: 0.0261


[0.0012434563832357526, 0.8206272125244141, 0.02613619528710842]

### 2002

In [16]:
from sklearn.preprocessing import StandardScaler
year = 2002
data_train = pd.read_csv(os.path.join('D:/SGYL/SM_results_data/Train/split/','train_data_'+str(year)+'.csv'))
data_validate = pd.read_csv(os.path.join('D:/SGYL/SM_results_data/Train/split/','validate_data_'+str(year)+'.csv'))
data_test = pd.read_csv(os.path.join('D:/SGYL/SM_results_data/Train/split/','test_data_'+str(year)+'.csv'))

X_train = data_train.drop(['SM'],axis = 1)
y_train = data_train['SM'].copy()   
X_validate = data_validate.drop(['SM'],axis = 1)
y_validate = data_validate['SM'].copy()
X_test = data_test.drop(['SM'],axis = 1)
y_test = data_test['SM'].copy()

standarder = StandardScaler()
X_train = standarder.fit_transform(X_train)
X_validate = standarder.transform(X_validate)
X_test = standarder.transform(X_test)

X_train = X_train.reshape(X_train.shape[0],X_train.shape[1],1)
X_validate = X_validate.reshape(X_validate.shape[0],X_validate.shape[1],1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1],1)

print('X_train shape:',X_train.shape)
print('X_validate shape:',X_validate.shape)

X_train shape: (505950, 19, 1)
X_validate shape: (151785, 19, 1)


In [17]:

def scheduler(epoch,lr):
    # 每隔30个epoch，学习率减小为原来的1/10
    if epoch % 30 == 0 and epoch != 0:
        print("lr changed to {}".format(lr * 0.1))
        return lr * 0.1
    else :
        return lr

callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=20,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
hypermodel = MyHyperModel()
tuner = keras_tuner.BayesianOptimization(
            hypermodel,
            objective=keras_tuner.Objective('val_R2', direction="max"),
            num_initial_points=50,
            max_trials=max_trials,
            overwrite = True,
            directory='D:/SGYL/SM_results_data/Bayesian_Opt/Full_ResNet/',
            project_name=('Full_ResNet_'+str(year)))
tuner.search(X_train,y_train,epochs=100,batch_size = 4096,validation_data=(X_validate,y_validate),shuffle = True,callbacks = [reduce_lr,callback_early_stopping],verbose = 2)

Trial 50 Complete [00h 02m 39s]
val_R2: 0.23078691959381104

Best val_R2 So Far: 0.8442344665527344
Total elapsed time: 02h 04m 47s
INFO:tensorflow:Oracle triggered exit


In [18]:
filepath="Full_ResNet_"+str(year)+"_best.hdf5"
callback_checkpoints = keras.callbacks.ModelCheckpoint(os.path.join('D:/SGYL/SM_results_data/check_points/Full_ResNet/',filepath),monitor='val_loss',save_best_only=True,verbose=1)
callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=50,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
callbacks = [reduce_lr,callback_early_stopping,callback_checkpoints]

if training_mode == 'model':
    model_best = tuner.get_best_models()[0]
else :
    model_best = tuner.hypermodel.build(tuner.get_best_hyperparameters()[0])
model_best.fit(X_train,y_train,epochs=300,batch_size = 4096,validation_data=(X_validate,y_validate),shuffle = True,callbacks = callbacks,verbose = 2)

Epoch 1/300

Epoch 00001: val_loss improved from inf to 0.00108, saving model to D:/SGYL/SM_results_data/check_points/Full_ResNet\Full_ResNet_2002_best.hdf5
124/124 - 2s - loss: 9.5195e-04 - R2: 0.8636 - mae: 0.0227 - val_loss: 0.0011 - val_R2: 0.8443 - val_mae: 0.0240
Epoch 2/300

Epoch 00002: val_loss improved from 0.00108 to 0.00108, saving model to D:/SGYL/SM_results_data/check_points/Full_ResNet\Full_ResNet_2002_best.hdf5
124/124 - 2s - loss: 9.5089e-04 - R2: 0.8637 - mae: 0.0227 - val_loss: 0.0011 - val_R2: 0.8443 - val_mae: 0.0240
Epoch 3/300

Epoch 00003: val_loss did not improve from 0.00108
124/124 - 2s - loss: 9.5023e-04 - R2: 0.8638 - mae: 0.0227 - val_loss: 0.0011 - val_R2: 0.8441 - val_mae: 0.0240
Epoch 4/300

Epoch 00004: val_loss did not improve from 0.00108
124/124 - 2s - loss: 9.5005e-04 - R2: 0.8639 - mae: 0.0227 - val_loss: 0.0011 - val_R2: 0.8438 - val_mae: 0.0240
Epoch 5/300

Epoch 00005: val_loss did not improve from 0.00108
124/124 - 1s - loss: 9.5109e-04 - R2: 

In [19]:
#save model
save_path = os.path.join('D:/SGYL/SM_results_data/model/Full_ResNet/','Full_ResNet_'+str(year)+'.hdf5')
model_best.save(save_path)
if (os.path.exists(save_path)):
    print('save model for year:',year)

#save model config
import json
bestConfig=tuner.get_best_hyperparameters()[0].get_config()
f = open(os.path.join('D:/SGYL/SM_results_data/Bayesian_Opt/Full_ResNet/Best_Config/','Full_ResNet_'+str(year)+'.json'), 'w')
json.dump(bestConfig,f)
f.close()
print(bestConfig['values'])

model_best.evaluate(X_test,y_test)

save model for year: 2002
{'Nodes_choice': 7, 'activation': 'tanh', 'activate_dense_last': 'sigmoid', 'lr': 0.005594661103196282}
2033/2033 [==============================] - 6s 3ms/step - loss: 0.0011 - R2: 0.8325 - mae: 0.0241


[0.0011013131588697433, 0.8325309157371521, 0.0240787323564291]

### 2003

In [20]:
from sklearn.preprocessing import StandardScaler
year = 2003
data_train = pd.read_csv(os.path.join('D:/SGYL/SM_results_data/Train/split/','train_data_'+str(year)+'.csv'))
data_validate = pd.read_csv(os.path.join('D:/SGYL/SM_results_data/Train/split/','validate_data_'+str(year)+'.csv'))
data_test = pd.read_csv(os.path.join('D:/SGYL/SM_results_data/Train/split/','test_data_'+str(year)+'.csv'))

X_train = data_train.drop(['SM'],axis = 1)
y_train = data_train['SM'].copy()   
X_validate = data_validate.drop(['SM'],axis = 1)
y_validate = data_validate['SM'].copy()
X_test = data_test.drop(['SM'],axis = 1)
y_test = data_test['SM'].copy()

standarder = StandardScaler()
X_train = standarder.fit_transform(X_train)
X_validate = standarder.transform(X_validate)
X_test = standarder.transform(X_test)

X_train = X_train.reshape(X_train.shape[0],X_train.shape[1],1)
X_validate = X_validate.reshape(X_validate.shape[0],X_validate.shape[1],1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1],1)

print('X_train shape:',X_train.shape)
print('X_validate shape:',X_validate.shape)

X_train shape: (507932, 19, 1)
X_validate shape: (152380, 19, 1)


In [21]:

def scheduler(epoch,lr):
    # 每隔30个epoch，学习率减小为原来的1/10
    if epoch % 30 == 0 and epoch != 0:
        print("lr changed to {}".format(lr * 0.1))
        return lr * 0.1
    else :
        return lr

callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=20,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
hypermodel = MyHyperModel()
tuner = keras_tuner.BayesianOptimization(
            hypermodel,
            objective=keras_tuner.Objective('val_R2', direction="max"),
            num_initial_points=50,
            max_trials=max_trials,
            overwrite = True,
            directory='D:/SGYL/SM_results_data/Bayesian_Opt/Full_ResNet/',
            project_name=('Full_ResNet_'+str(year)))
tuner.search(X_train,y_train,epochs=100,batch_size = 4096,validation_data=(X_validate,y_validate),shuffle = True,callbacks = [reduce_lr,callback_early_stopping],verbose = 2)
def scheduler(epoch,lr):
    # 每隔30个epoch，学习率减小为原来的1/10
    if epoch % 30 == 0 and epoch != 0:
        print("lr changed to {}".format(lr * 0.1))
        return lr * 0.1
    else :
        return lr

callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=20,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
hypermodel = MyHyperModel()
tuner = keras_tuner.BayesianOptimization(
            hypermodel,
            objective=keras_tuner.Objective('val_R2', direction="max"),
            num_initial_points=50,
            max_trials=max_trials,
            overwrite = True,
            directory='D:/SGYL/SM_results_data/Bayesian_Opt/Full_ResNet/',
            project_name=('Full_ResNet_'+str(year)))
tuner.search(X_train,y_train,epochs=100,batch_size = 4096,validation_data=(X_validate,y_validate),shuffle = True,callbacks = [reduce_lr,callback_early_stopping],verbose = 2)

Trial 50 Complete [00h 01m 47s]
val_R2: 0.7651962041854858

Best val_R2 So Far: 0.8649402260780334
Total elapsed time: 01h 47m 38s
INFO:tensorflow:Oracle triggered exit


In [22]:
filepath="Full_ResNet_"+str(year)+"_best.hdf5"
callback_checkpoints = keras.callbacks.ModelCheckpoint(os.path.join('D:/SGYL/SM_results_data/check_points/Full_ResNet/',filepath),monitor='val_loss',save_best_only=True,verbose=1)
callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=50,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
callbacks = [reduce_lr,callback_early_stopping,callback_checkpoints]

if training_mode == 'model':
    model_best = tuner.get_best_models()[0]
else :
    model_best = tuner.hypermodel.build(tuner.get_best_hyperparameters()[0])
model_best.fit(X_train,y_train,epochs=300,batch_size = 4096,validation_data=(X_validate,y_validate),shuffle = True,callbacks = callbacks,verbose = 2)

Epoch 1/300

Epoch 00001: val_loss improved from inf to 0.00102, saving model to D:/SGYL/SM_results_data/check_points/Full_ResNet\Full_ResNet_2003_best.hdf5
125/125 - 2s - loss: 9.0804e-04 - R2: 0.8670 - mae: 0.0222 - val_loss: 0.0010 - val_R2: 0.8522 - val_mae: 0.0235
Epoch 2/300

Epoch 00002: val_loss did not improve from 0.00102
125/125 - 2s - loss: 9.0769e-04 - R2: 0.8661 - mae: 0.0222 - val_loss: 0.0011 - val_R2: 0.8409 - val_mae: 0.0245
Epoch 3/300

Epoch 00003: val_loss improved from 0.00102 to 0.00099, saving model to D:/SGYL/SM_results_data/check_points/Full_ResNet\Full_ResNet_2003_best.hdf5
125/125 - 2s - loss: 9.0909e-04 - R2: 0.8648 - mae: 0.0222 - val_loss: 9.8639e-04 - val_R2: 0.8570 - val_mae: 0.0232
Epoch 4/300

Epoch 00004: val_loss did not improve from 0.00099
125/125 - 2s - loss: 9.0597e-04 - R2: 0.8670 - mae: 0.0222 - val_loss: 0.0010 - val_R2: 0.8526 - val_mae: 0.0236
Epoch 5/300

Epoch 00005: val_loss did not improve from 0.00099
125/125 - 1s - loss: 9.0472e-04 - 

In [23]:
#save model
save_path = os.path.join('D:/SGYL/SM_results_data/model/Full_ResNet/','Full_ResNet_'+str(year)+'.hdf5')
model_best.save(save_path)
if (os.path.exists(save_path)):
    print('save model for year:',year)

#save model config
import json
bestConfig=tuner.get_best_hyperparameters()[0].get_config()
f = open(os.path.join('D:/SGYL/SM_results_data/Bayesian_Opt/Full_ResNet/Best_Config/','Full_ResNet_'+str(year)+'.json'), 'w')
json.dump(bestConfig,f)
f.close()
print(bestConfig['values'])

model_best.evaluate(X_test,y_test)

save model for year: 2003
{'Nodes_choice': 7, 'activation': 'tanh', 'activate_dense_last': 'sigmoid', 'lr': 0.0051059346908493425}
2041/2041 [==============================] - 6s 3ms/step - loss: 0.0010 - R2: 0.8433 - mae: 0.0236


[0.001030686660669744, 0.8433122038841248, 0.023618338629603386]

### 2004

In [24]:
from sklearn.preprocessing import StandardScaler
year = 2004
data_train = pd.read_csv(os.path.join('D:/SGYL/SM_results_data/Train/split/','train_data_'+str(year)+'.csv'))
data_validate = pd.read_csv(os.path.join('D:/SGYL/SM_results_data/Train/split/','validate_data_'+str(year)+'.csv'))
data_test = pd.read_csv(os.path.join('D:/SGYL/SM_results_data/Train/split/','test_data_'+str(year)+'.csv'))

X_train = data_train.drop(['SM'],axis = 1)
y_train = data_train['SM'].copy()   
X_validate = data_validate.drop(['SM'],axis = 1)
y_validate = data_validate['SM'].copy()
X_test = data_test.drop(['SM'],axis = 1)
y_test = data_test['SM'].copy()

standarder = StandardScaler()
X_train = standarder.fit_transform(X_train)
X_validate = standarder.transform(X_validate)
X_test = standarder.transform(X_test)

X_train = X_train.reshape(X_train.shape[0],X_train.shape[1],1)
X_validate = X_validate.reshape(X_validate.shape[0],X_validate.shape[1],1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1],1)

print('X_train shape:',X_train.shape)
print('X_validate shape:',X_validate.shape)

X_train shape: (500833, 19, 1)
X_validate shape: (150250, 19, 1)


In [25]:

def scheduler(epoch,lr):
    # 每隔30个epoch，学习率减小为原来的1/10
    if epoch % 30 == 0 and epoch != 0:
        print("lr changed to {}".format(lr * 0.1))
        return lr * 0.1
    else :
        return lr

callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=20,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
hypermodel = MyHyperModel()
tuner = keras_tuner.BayesianOptimization(
            hypermodel,
            objective=keras_tuner.Objective('val_R2', direction="max"),
            num_initial_points=50,
            max_trials=max_trials,
            overwrite = True,
            directory='D:/SGYL/SM_results_data/Bayesian_Opt/Full_ResNet/',
            project_name=('Full_ResNet_'+str(year)))
tuner.search(X_train,y_train,epochs=100,batch_size = 4096,validation_data=(X_validate,y_validate),shuffle = True,callbacks = [reduce_lr,callback_early_stopping],verbose = 2)

Trial 50 Complete [00h 02m 37s]
val_R2: 0.7537385821342468

Best val_R2 So Far: 0.865343451499939
Total elapsed time: 02h 05m 00s
INFO:tensorflow:Oracle triggered exit


In [26]:
filepath="Full_ResNet_"+str(year)+"_best.hdf5"
callback_checkpoints = keras.callbacks.ModelCheckpoint(os.path.join('D:/SGYL/SM_results_data/check_points/Full_ResNet/',filepath),monitor='val_loss',save_best_only=True,verbose=1)
callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=50,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
callbacks = [reduce_lr,callback_early_stopping,callback_checkpoints]

if training_mode == 'model':
    model_best = tuner.get_best_models()[0]
else :
    model_best = tuner.hypermodel.build(tuner.get_best_hyperparameters()[0])
model_best.fit(X_train,y_train,epochs=300,batch_size = 4096,validation_data=(X_validate,y_validate),shuffle = True,callbacks = callbacks,verbose = 2)

Epoch 1/300

Epoch 00001: val_loss improved from inf to 0.00094, saving model to D:/SGYL/SM_results_data/check_points/Full_ResNet\Full_ResNet_2004_best.hdf5
123/123 - 2s - loss: 8.2841e-04 - R2: 0.8804 - mae: 0.0212 - val_loss: 9.3521e-04 - val_R2: 0.8650 - val_mae: 0.0222
Epoch 2/300

Epoch 00002: val_loss improved from 0.00094 to 0.00093, saving model to D:/SGYL/SM_results_data/check_points/Full_ResNet\Full_ResNet_2004_best.hdf5
123/123 - 2s - loss: 8.2830e-04 - R2: 0.8804 - mae: 0.0212 - val_loss: 9.3330e-04 - val_R2: 0.8652 - val_mae: 0.0222
Epoch 3/300

Epoch 00003: val_loss did not improve from 0.00093
123/123 - 1s - loss: 8.2977e-04 - R2: 0.8804 - mae: 0.0212 - val_loss: 9.3469e-04 - val_R2: 0.8650 - val_mae: 0.0223
Epoch 4/300

Epoch 00004: val_loss did not improve from 0.00093
123/123 - 1s - loss: 8.2972e-04 - R2: 0.8802 - mae: 0.0212 - val_loss: 9.4077e-04 - val_R2: 0.8642 - val_mae: 0.0223
Epoch 5/300

Epoch 00005: val_loss did not improve from 0.00093
123/123 - 1s - loss: 8

In [27]:
#save model
save_path = os.path.join('D:/SGYL/SM_results_data/model/Full_ResNet/','Full_ResNet_'+str(year)+'.hdf5')
model_best.save(save_path)
if (os.path.exists(save_path)):
    print('save model for year:',year)

#save model config
import json
bestConfig=tuner.get_best_hyperparameters()[0].get_config()
f = open(os.path.join('D:/SGYL/SM_results_data/Bayesian_Opt/Full_ResNet/Best_Config/','Full_ResNet_'+str(year)+'.json'), 'w')
json.dump(bestConfig,f)
f.close()
print(bestConfig['values'])

model_best.evaluate(X_test,y_test)

save model for year: 2004
{'Nodes_choice': 3, 'activation': 'tanh', 'activate_dense_last': 'sigmoid', 'lr': 0.0033117989675966585}
2013/2013 [==============================] - 5s 3ms/step - loss: 9.0938e-04 - R2: 0.8586 - mae: 0.0221


[0.0009093779954127967, 0.8585960865020752, 0.022144010290503502]

### 2005

In [28]:
from sklearn.preprocessing import StandardScaler
year = 2005
data_train = pd.read_csv(os.path.join('D:/SGYL/SM_results_data/Train/split/','train_data_'+str(year)+'.csv'))
data_validate = pd.read_csv(os.path.join('D:/SGYL/SM_results_data/Train/split/','validate_data_'+str(year)+'.csv'))
data_test = pd.read_csv(os.path.join('D:/SGYL/SM_results_data/Train/split/','test_data_'+str(year)+'.csv'))

X_train = data_train.drop(['SM'],axis = 1)
y_train = data_train['SM'].copy()   
X_validate = data_validate.drop(['SM'],axis = 1)
y_validate = data_validate['SM'].copy()
X_test = data_test.drop(['SM'],axis = 1)
y_test = data_test['SM'].copy()

standarder = StandardScaler()
X_train = standarder.fit_transform(X_train)
X_validate = standarder.transform(X_validate)
X_test = standarder.transform(X_test)

X_train = X_train.reshape(X_train.shape[0],X_train.shape[1],1)
X_validate = X_validate.reshape(X_validate.shape[0],X_validate.shape[1],1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1],1)

print('X_train shape:',X_train.shape)
print('X_validate shape:',X_validate.shape)

X_train shape: (505351, 19, 1)
X_validate shape: (151606, 19, 1)


In [29]:

def scheduler(epoch,lr):
    # 每隔30个epoch，学习率减小为原来的1/10
    if epoch % 30 == 0 and epoch != 0:
        print("lr changed to {}".format(lr * 0.1))
        return lr * 0.1
    else :
        return lr

callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=20,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
hypermodel = MyHyperModel()
tuner = keras_tuner.BayesianOptimization(
            hypermodel,
            objective=keras_tuner.Objective('val_R2', direction="max"),
            num_initial_points=50,
            max_trials=max_trials,
            overwrite = True,
            directory='D:/SGYL/SM_results_data/Bayesian_Opt/Full_ResNet/',
            project_name=('Full_ResNet_'+str(year)))
tuner.search(X_train,y_train,epochs=100,batch_size = 4096,validation_data=(X_validate,y_validate),shuffle = True,callbacks = [reduce_lr,callback_early_stopping],verbose = 2)

Trial 50 Complete [00h 02m 38s]
val_R2: 0.8054718971252441

Best val_R2 So Far: 0.8770922422409058
Total elapsed time: 01h 56m 33s
INFO:tensorflow:Oracle triggered exit


In [30]:
filepath="Full_ResNet_"+str(year)+"_best.hdf5"
callback_checkpoints = keras.callbacks.ModelCheckpoint(os.path.join('D:/SGYL/SM_results_data/check_points/Full_ResNet/',filepath),monitor='val_loss',save_best_only=True,verbose=1)
callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=50,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
callbacks = [reduce_lr,callback_early_stopping,callback_checkpoints]

if training_mode == 'model':
    model_best = tuner.get_best_models()[0]
else :
    model_best = tuner.hypermodel.build(tuner.get_best_hyperparameters()[0])
model_best.fit(X_train,y_train,epochs=300,batch_size = 4096,validation_data=(X_validate,y_validate),shuffle = True,callbacks = callbacks,verbose = 2)

Epoch 1/300

Epoch 00001: val_loss improved from inf to 0.00087, saving model to D:/SGYL/SM_results_data/check_points/Full_ResNet\Full_ResNet_2005_best.hdf5
124/124 - 2s - loss: 7.8619e-04 - R2: 0.8886 - mae: 0.0206 - val_loss: 8.6771e-04 - val_R2: 0.8770 - val_mae: 0.0216
Epoch 2/300

Epoch 00002: val_loss did not improve from 0.00087
124/124 - 2s - loss: 7.8873e-04 - R2: 0.8882 - mae: 0.0207 - val_loss: 8.7015e-04 - val_R2: 0.8766 - val_mae: 0.0217
Epoch 3/300

Epoch 00003: val_loss did not improve from 0.00087
124/124 - 2s - loss: 7.8651e-04 - R2: 0.8885 - mae: 0.0206 - val_loss: 8.6909e-04 - val_R2: 0.8767 - val_mae: 0.0217
Epoch 4/300

Epoch 00004: val_loss improved from 0.00087 to 0.00087, saving model to D:/SGYL/SM_results_data/check_points/Full_ResNet\Full_ResNet_2005_best.hdf5
124/124 - 2s - loss: 7.8817e-04 - R2: 0.8883 - mae: 0.0207 - val_loss: 8.6564e-04 - val_R2: 0.8772 - val_mae: 0.0216
Epoch 5/300

Epoch 00005: val_loss did not improve from 0.00087
124/124 - 2s - loss: 7

In [31]:
#save model
save_path = os.path.join('D:/SGYL/SM_results_data/model/Full_ResNet/','Full_ResNet_'+str(year)+'.hdf5')
model_best.save(save_path)
if (os.path.exists(save_path)):
    print('save model for year:',year)

#save model config
import json
bestConfig=tuner.get_best_hyperparameters()[0].get_config()
f = open(os.path.join('D:/SGYL/SM_results_data/Bayesian_Opt/Full_ResNet/Best_Config/','Full_ResNet_'+str(year)+'.json'), 'w')
json.dump(bestConfig,f)
f.close()
print(bestConfig['values'])

model_best.evaluate(X_test,y_test)

save model for year: 2005
{'Nodes_choice': 7, 'activation': 'tanh', 'activate_dense_last': 'sigmoid', 'lr': 0.0068286015611718626}
2031/2031 [==============================] - 7s 3ms/step - loss: 8.8717e-04 - R2: 0.8660 - mae: 0.0218


[0.0008871700847521424, 0.8660190105438232, 0.021811019629240036]

### 2006

In [32]:
from sklearn.preprocessing import StandardScaler
year = 2006
data_train = pd.read_csv(os.path.join('D:/SGYL/SM_results_data/Train/split/','train_data_'+str(year)+'.csv'))
data_validate = pd.read_csv(os.path.join('D:/SGYL/SM_results_data/Train/split/','validate_data_'+str(year)+'.csv'))
data_test = pd.read_csv(os.path.join('D:/SGYL/SM_results_data/Train/split/','test_data_'+str(year)+'.csv'))

X_train = data_train.drop(['SM'],axis = 1)
y_train = data_train['SM'].copy()   
X_validate = data_validate.drop(['SM'],axis = 1)
y_validate = data_validate['SM'].copy()
X_test = data_test.drop(['SM'],axis = 1)
y_test = data_test['SM'].copy()

standarder = StandardScaler()
X_train = standarder.fit_transform(X_train)
X_validate = standarder.transform(X_validate)
X_test = standarder.transform(X_test)

X_train = X_train.reshape(X_train.shape[0],X_train.shape[1],1)
X_validate = X_validate.reshape(X_validate.shape[0],X_validate.shape[1],1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1],1)

print('X_train shape:',X_train.shape)
print('X_validate shape:',X_validate.shape)

X_train shape: (513265, 19, 1)
X_validate shape: (153980, 19, 1)


In [33]:

def scheduler(epoch,lr):
    # 每隔30个epoch，学习率减小为原来的1/10
    if epoch % 30 == 0 and epoch != 0:
        print("lr changed to {}".format(lr * 0.1))
        return lr * 0.1
    else :
        return lr

callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=20,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
hypermodel = MyHyperModel()
tuner = keras_tuner.BayesianOptimization(
            hypermodel,
            objective=keras_tuner.Objective('val_R2', direction="max"),
            num_initial_points=50,
            max_trials=max_trials,
            overwrite = True,
            directory='D:/SGYL/SM_results_data/Bayesian_Opt/Full_ResNet/',
            project_name=('Full_ResNet_'+str(year)))
tuner.search(X_train,y_train,epochs=100,batch_size = 4096,validation_data=(X_validate,y_validate),shuffle = True,callbacks = [reduce_lr,callback_early_stopping],verbose = 2)

Trial 50 Complete [00h 03m 06s]
val_R2: 0.8122607469558716

Best val_R2 So Far: 0.8436573147773743
Total elapsed time: 02h 14m 59s


INFO:tensorflow:Oracle triggered exit


In [34]:
filepath="Full_ResNet_"+str(year)+"_best.hdf5"
callback_checkpoints = keras.callbacks.ModelCheckpoint(os.path.join('D:/SGYL/SM_results_data/check_points/Full_ResNet/',filepath),monitor='val_loss',save_best_only=True,verbose=1)
callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=50,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
callbacks = [reduce_lr,callback_early_stopping,callback_checkpoints]

if training_mode == 'model':
    model_best = tuner.get_best_models()[0]
else :
    model_best = tuner.hypermodel.build(tuner.get_best_hyperparameters()[0])
model_best.fit(X_train,y_train,epochs=300,batch_size = 4096,validation_data=(X_validate,y_validate),shuffle = True,callbacks = callbacks,verbose = 2)

Epoch 1/300

Epoch 00001: val_loss improved from inf to 0.00105, saving model to D:/SGYL/SM_results_data/check_points/Full_ResNet\Full_ResNet_2006_best.hdf5
126/126 - 2s - loss: 9.9437e-04 - R2: 0.8521 - mae: 0.0234 - val_loss: 0.0010 - val_R2: 0.8436 - val_mae: 0.0238
Epoch 2/300

Epoch 00002: val_loss did not improve from 0.00105
126/126 - 2s - loss: 9.9565e-04 - R2: 0.8517 - mae: 0.0234 - val_loss: 0.0011 - val_R2: 0.8435 - val_mae: 0.0238
Epoch 3/300

Epoch 00003: val_loss did not improve from 0.00105
126/126 - 2s - loss: 9.9618e-04 - R2: 0.8517 - mae: 0.0234 - val_loss: 0.0011 - val_R2: 0.8430 - val_mae: 0.0239
Epoch 4/300

Epoch 00004: val_loss did not improve from 0.00105
126/126 - 2s - loss: 9.9492e-04 - R2: 0.8519 - mae: 0.0233 - val_loss: 0.0011 - val_R2: 0.8435 - val_mae: 0.0238
Epoch 5/300

Epoch 00005: val_loss did not improve from 0.00105
126/126 - 1s - loss: 9.9466e-04 - R2: 0.8520 - mae: 0.0234 - val_loss: 0.0011 - val_R2: 0.8435 - val_mae: 0.0238
Epoch 6/300

Epoch 000

In [35]:
#save model
save_path = os.path.join('D:/SGYL/SM_results_data/model/Full_ResNet/','Full_ResNet_'+str(year)+'.hdf5')
model_best.save(save_path)
if (os.path.exists(save_path)):
    print('save model for year:',year)

#save model config
import json
bestConfig=tuner.get_best_hyperparameters()[0].get_config()
f = open(os.path.join('D:/SGYL/SM_results_data/Bayesian_Opt/Full_ResNet/Best_Config/','Full_ResNet_'+str(year)+'.json'), 'w')
json.dump(bestConfig,f)
f.close()
print(bestConfig['values'])

model_best.evaluate(X_test,y_test)

save model for year: 2006
{'Nodes_choice': 5, 'activation': 'tanh', 'activate_dense_last': 'sigmoid', 'lr': 0.006913157533608411}
2063/2063 [==============================] - 6s 3ms/step - loss: 0.0011 - R2: 0.8333 - mae: 0.0240


[0.0010520952055230737, 0.8332618474960327, 0.024010831490159035]

### 2007

In [36]:
from sklearn.preprocessing import StandardScaler
year = 2007
data_train = pd.read_csv(os.path.join('D:/SGYL/SM_results_data/Train/split/','train_data_'+str(year)+'.csv'))
data_validate = pd.read_csv(os.path.join('D:/SGYL/SM_results_data/Train/split/','validate_data_'+str(year)+'.csv'))
data_test = pd.read_csv(os.path.join('D:/SGYL/SM_results_data/Train/split/','test_data_'+str(year)+'.csv'))

X_train = data_train.drop(['SM'],axis = 1)
y_train = data_train['SM'].copy()   
X_validate = data_validate.drop(['SM'],axis = 1)
y_validate = data_validate['SM'].copy()
X_test = data_test.drop(['SM'],axis = 1)
y_test = data_test['SM'].copy()

standarder = StandardScaler()
X_train = standarder.fit_transform(X_train)
X_validate = standarder.transform(X_validate)
X_test = standarder.transform(X_test)

X_train = X_train.reshape(X_train.shape[0],X_train.shape[1],1)
X_validate = X_validate.reshape(X_validate.shape[0],X_validate.shape[1],1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1],1)

print('X_train shape:',X_train.shape)
print('X_validate shape:',X_validate.shape)

X_train shape: (618216, 19, 1)
X_validate shape: (185465, 19, 1)


In [37]:

def scheduler(epoch,lr):
    # 每隔30个epoch，学习率减小为原来的1/10
    if epoch % 30 == 0 and epoch != 0:
        print("lr changed to {}".format(lr * 0.1))
        return lr * 0.1
    else :
        return lr

callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=20,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
hypermodel = MyHyperModel()
tuner = keras_tuner.BayesianOptimization(
            hypermodel,
            objective=keras_tuner.Objective('val_R2', direction="max"),
            num_initial_points=50,
            max_trials=max_trials,
            overwrite = True,
            directory='D:/SGYL/SM_results_data/Bayesian_Opt/Full_ResNet/',
            project_name=('Full_ResNet_'+str(year)))
tuner.search(X_train,y_train,epochs=100,batch_size = 4096,validation_data=(X_validate,y_validate),shuffle = True,callbacks = [reduce_lr,callback_early_stopping],verbose = 2)

Trial 50 Complete [00h 03m 28s]
val_R2: 0.829420268535614

Best val_R2 So Far: 0.8785305619239807
Total elapsed time: 02h 40m 11s
INFO:tensorflow:Oracle triggered exit


In [38]:
filepath="Full_ResNet_"+str(year)+"_best.hdf5"
callback_checkpoints = keras.callbacks.ModelCheckpoint(os.path.join('D:/SGYL/SM_results_data/check_points/Full_ResNet/',filepath),monitor='val_loss',save_best_only=True,verbose=1)
callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=50,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
callbacks = [reduce_lr,callback_early_stopping,callback_checkpoints]

if training_mode == 'model':
    model_best = tuner.get_best_models()[0]
else :
    model_best = tuner.hypermodel.build(tuner.get_best_hyperparameters()[0])
model_best.fit(X_train,y_train,epochs=300,batch_size = 4096,validation_data=(X_validate,y_validate),shuffle = True,callbacks = callbacks,verbose = 2)

Epoch 1/300

Epoch 00001: val_loss improved from inf to 0.00093, saving model to D:/SGYL/SM_results_data/check_points/Full_ResNet\Full_ResNet_2007_best.hdf5
151/151 - 2s - loss: 8.1775e-04 - R2: 0.8927 - mae: 0.0212 - val_loss: 9.2536e-04 - val_R2: 0.8784 - val_mae: 0.0224
Epoch 2/300

Epoch 00002: val_loss did not improve from 0.00093
151/151 - 2s - loss: 8.2074e-04 - R2: 0.8923 - mae: 0.0212 - val_loss: 9.2599e-04 - val_R2: 0.8783 - val_mae: 0.0224
Epoch 3/300

Epoch 00003: val_loss did not improve from 0.00093
151/151 - 2s - loss: 8.2004e-04 - R2: 0.8924 - mae: 0.0212 - val_loss: 9.2638e-04 - val_R2: 0.8782 - val_mae: 0.0224
Epoch 4/300

Epoch 00004: val_loss improved from 0.00093 to 0.00092, saving model to D:/SGYL/SM_results_data/check_points/Full_ResNet\Full_ResNet_2007_best.hdf5
151/151 - 2s - loss: 8.1707e-04 - R2: 0.8928 - mae: 0.0212 - val_loss: 9.2495e-04 - val_R2: 0.8784 - val_mae: 0.0224
Epoch 5/300

Epoch 00005: val_loss did not improve from 0.00092
151/151 - 2s - loss: 8

In [39]:
#save model
save_path = os.path.join('D:/SGYL/SM_results_data/model/Full_ResNet/','Full_ResNet_'+str(year)+'.hdf5')
model_best.save(save_path)
if (os.path.exists(save_path)):
    print('save model for year:',year)

#save model config
import json
bestConfig=tuner.get_best_hyperparameters()[0].get_config()
f = open(os.path.join('D:/SGYL/SM_results_data/Bayesian_Opt/Full_ResNet/Best_Config/','Full_ResNet_'+str(year)+'.json'), 'w')
json.dump(bestConfig,f)
f.close()
print(bestConfig['values'])

model_best.evaluate(X_test,y_test)

save model for year: 2007
{'Nodes_choice': 3, 'activation': 'tanh', 'activate_dense_last': 'sigmoid', 'lr': 0.006809374456390052}
2484/2484 [==============================] - 6s 2ms/step - loss: 9.2949e-04 - R2: 0.8701 - mae: 0.0224


[0.0009294944466091692, 0.8701308369636536, 0.02237297222018242]

### 2008

In [40]:
from sklearn.preprocessing import StandardScaler
year = 2008
data_train = pd.read_csv(os.path.join('D:/SGYL/SM_results_data/Train/split/','train_data_'+str(year)+'.csv'))
data_validate = pd.read_csv(os.path.join('D:/SGYL/SM_results_data/Train/split/','validate_data_'+str(year)+'.csv'))
data_test = pd.read_csv(os.path.join('D:/SGYL/SM_results_data/Train/split/','test_data_'+str(year)+'.csv'))

X_train = data_train.drop(['SM'],axis = 1)
y_train = data_train['SM'].copy()   
X_validate = data_validate.drop(['SM'],axis = 1)
y_validate = data_validate['SM'].copy()
X_test = data_test.drop(['SM'],axis = 1)
y_test = data_test['SM'].copy()

standarder = StandardScaler()
X_train = standarder.fit_transform(X_train)
X_validate = standarder.transform(X_validate)
X_test = standarder.transform(X_test)

X_train = X_train.reshape(X_train.shape[0],X_train.shape[1],1)
X_validate = X_validate.reshape(X_validate.shape[0],X_validate.shape[1],1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1],1)

print('X_train shape:',X_train.shape)
print('X_validate shape:',X_validate.shape)

X_train shape: (625440, 19, 1)
X_validate shape: (187632, 19, 1)


In [41]:

def scheduler(epoch,lr):
    # 每隔30个epoch，学习率减小为原来的1/10
    if epoch % 30 == 0 and epoch != 0:
        print("lr changed to {}".format(lr * 0.1))
        return lr * 0.1
    else :
        return lr

callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=20,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
hypermodel = MyHyperModel()
tuner = keras_tuner.BayesianOptimization(
            hypermodel,
            objective=keras_tuner.Objective('val_R2', direction="max"),
            num_initial_points=50,
            max_trials=max_trials,
            overwrite = True,
            directory='D:/SGYL/SM_results_data/Bayesian_Opt/Full_ResNet/',
            project_name=('Full_ResNet_'+str(year)))
tuner.search(X_train,y_train,epochs=100,batch_size = 4096,validation_data=(X_validate,y_validate),shuffle = True,callbacks = [reduce_lr,callback_early_stopping],verbose = 2)

Trial 50 Complete [00h 03m 33s]
val_R2: 0.8552377820014954

Best val_R2 So Far: 0.8713366389274597
Total elapsed time: 02h 36m 38s
INFO:tensorflow:Oracle triggered exit


In [42]:
filepath="Full_ResNet_"+str(year)+"_best.hdf5"
callback_checkpoints = keras.callbacks.ModelCheckpoint(os.path.join('D:/SGYL/SM_results_data/check_points/Full_ResNet/',filepath),monitor='val_loss',save_best_only=True,verbose=1)
callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=50,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
callbacks = [reduce_lr,callback_early_stopping,callback_checkpoints]

if training_mode == 'model':
    model_best = tuner.get_best_models()[0]
else :
    model_best = tuner.hypermodel.build(tuner.get_best_hyperparameters()[0])
model_best.fit(X_train,y_train,epochs=300,batch_size = 4096,validation_data=(X_validate,y_validate),shuffle = True,callbacks = callbacks,verbose = 2)

Epoch 1/300

Epoch 00001: val_loss improved from inf to 0.00100, saving model to D:/SGYL/SM_results_data/check_points/Full_ResNet\Full_ResNet_2008_best.hdf5
153/153 - 2s - loss: 9.8711e-04 - R2: 0.8735 - mae: 0.0234 - val_loss: 0.0010 - val_R2: 0.8711 - val_mae: 0.0236
Epoch 2/300

Epoch 00002: val_loss improved from 0.00100 to 0.00100, saving model to D:/SGYL/SM_results_data/check_points/Full_ResNet\Full_ResNet_2008_best.hdf5
153/153 - 2s - loss: 9.8752e-04 - R2: 0.8734 - mae: 0.0234 - val_loss: 0.0010 - val_R2: 0.8711 - val_mae: 0.0235
Epoch 3/300

Epoch 00003: val_loss improved from 0.00100 to 0.00100, saving model to D:/SGYL/SM_results_data/check_points/Full_ResNet\Full_ResNet_2008_best.hdf5
153/153 - 2s - loss: 9.8592e-04 - R2: 0.8736 - mae: 0.0233 - val_loss: 0.0010 - val_R2: 0.8713 - val_mae: 0.0235
Epoch 4/300

Epoch 00004: val_loss did not improve from 0.00100
153/153 - 2s - loss: 9.8673e-04 - R2: 0.8735 - mae: 0.0233 - val_loss: 0.0010 - val_R2: 0.8710 - val_mae: 0.0235
Epoch

In [43]:
#save model
save_path = os.path.join('D:/SGYL/SM_results_data/model/Full_ResNet/','Full_ResNet_'+str(year)+'.hdf5')
model_best.save(save_path)
if (os.path.exists(save_path)):
    print('save model for year:',year)

#save model config
import json
bestConfig=tuner.get_best_hyperparameters()[0].get_config()
f = open(os.path.join('D:/SGYL/SM_results_data/Bayesian_Opt/Full_ResNet/Best_Config/','Full_ResNet_'+str(year)+'.json'), 'w')
json.dump(bestConfig,f)
f.close()
print(bestConfig['values'])

model_best.evaluate(X_test,y_test)

save model for year: 2008
{'Nodes_choice': 4, 'activation': 'tanh', 'activate_dense_last': 'sigmoid', 'lr': 0.006423597973604284}
2513/2513 [==============================] - 7s 3ms/step - loss: 0.0010 - R2: 0.8619 - mae: 0.0236


[0.0010038731852546334, 0.8619005680084229, 0.02356559969484806]

### 2009

In [44]:
from sklearn.preprocessing import StandardScaler
year = 2009
data_train = pd.read_csv(os.path.join('D:/SGYL/SM_results_data/Train/split/','train_data_'+str(year)+'.csv'))
data_validate = pd.read_csv(os.path.join('D:/SGYL/SM_results_data/Train/split/','validate_data_'+str(year)+'.csv'))
data_test = pd.read_csv(os.path.join('D:/SGYL/SM_results_data/Train/split/','test_data_'+str(year)+'.csv'))

X_train = data_train.drop(['SM'],axis = 1)
y_train = data_train['SM'].copy()   
X_validate = data_validate.drop(['SM'],axis = 1)
y_validate = data_validate['SM'].copy()
X_test = data_test.drop(['SM'],axis = 1)
y_test = data_test['SM'].copy()

standarder = StandardScaler()
X_train = standarder.fit_transform(X_train)
X_validate = standarder.transform(X_validate)
X_test = standarder.transform(X_test)

X_train = X_train.reshape(X_train.shape[0],X_train.shape[1],1)
X_validate = X_validate.reshape(X_validate.shape[0],X_validate.shape[1],1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1],1)

print('X_train shape:',X_train.shape)
print('X_validate shape:',X_validate.shape)

X_train shape: (613756, 19, 1)
X_validate shape: (184127, 19, 1)


In [45]:

def scheduler(epoch,lr):
    # 每隔30个epoch，学习率减小为原来的1/10
    if epoch % 30 == 0 and epoch != 0:
        print("lr changed to {}".format(lr * 0.1))
        return lr * 0.1
    else :
        return lr

callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=20,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
hypermodel = MyHyperModel()
tuner = keras_tuner.BayesianOptimization(
            hypermodel,
            objective=keras_tuner.Objective('val_R2', direction="max"),
            num_initial_points=50,
            max_trials=max_trials,
            overwrite = True,
            directory='D:/SGYL/SM_results_data/Bayesian_Opt/Full_ResNet/',
            project_name=('Full_ResNet_'+str(year)))
tuner.search(X_train,y_train,epochs=100,batch_size = 4096,validation_data=(X_validate,y_validate),shuffle = True,callbacks = [reduce_lr,callback_early_stopping],verbose = 2)

Trial 50 Complete [00h 02m 56s]
val_R2: 0.7520607709884644

Best val_R2 So Far: 0.8801838159561157
Total elapsed time: 02h 34m 08s
INFO:tensorflow:Oracle triggered exit


In [46]:
filepath="Full_ResNet_"+str(year)+"_best.hdf5"
callback_checkpoints = keras.callbacks.ModelCheckpoint(os.path.join('D:/SGYL/SM_results_data/check_points/Full_ResNet/',filepath),monitor='val_loss',save_best_only=True,verbose=1)
callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=50,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
callbacks = [reduce_lr,callback_early_stopping,callback_checkpoints]

if training_mode == 'model':
    model_best = tuner.get_best_models()[0]
else :
    model_best = tuner.hypermodel.build(tuner.get_best_hyperparameters()[0])
model_best.fit(X_train,y_train,epochs=300,batch_size = 4096,validation_data=(X_validate,y_validate),shuffle = True,callbacks = callbacks,verbose = 2)

Epoch 1/300

Epoch 00001: val_loss improved from inf to 0.00089, saving model to D:/SGYL/SM_results_data/check_points/Full_ResNet\Full_ResNet_2009_best.hdf5
150/150 - 2s - loss: 8.5700e-04 - R2: 0.8852 - mae: 0.0215 - val_loss: 8.9231e-04 - val_R2: 0.8801 - val_mae: 0.0220
Epoch 2/300

Epoch 00002: val_loss improved from 0.00089 to 0.00089, saving model to D:/SGYL/SM_results_data/check_points/Full_ResNet\Full_ResNet_2009_best.hdf5
150/150 - 2s - loss: 8.5674e-04 - R2: 0.8852 - mae: 0.0215 - val_loss: 8.9160e-04 - val_R2: 0.8802 - val_mae: 0.0220
Epoch 3/300

Epoch 00003: val_loss did not improve from 0.00089
150/150 - 2s - loss: 8.5683e-04 - R2: 0.8852 - mae: 0.0216 - val_loss: 8.9171e-04 - val_R2: 0.8802 - val_mae: 0.0220
Epoch 4/300

Epoch 00004: val_loss did not improve from 0.00089
150/150 - 2s - loss: 8.5980e-04 - R2: 0.8848 - mae: 0.0216 - val_loss: 8.9213e-04 - val_R2: 0.8801 - val_mae: 0.0220
Epoch 5/300

Epoch 00005: val_loss did not improve from 0.00089
150/150 - 2s - loss: 8

In [47]:
#save model
save_path = os.path.join('D:/SGYL/SM_results_data/model/Full_ResNet/','Full_ResNet_'+str(year)+'.hdf5')
model_best.save(save_path)
if (os.path.exists(save_path)):
    print('save model for year:',year)

#save model config
import json
bestConfig=tuner.get_best_hyperparameters()[0].get_config()
f = open(os.path.join('D:/SGYL/SM_results_data/Bayesian_Opt/Full_ResNet/Best_Config/','Full_ResNet_'+str(year)+'.json'), 'w')
json.dump(bestConfig,f)
f.close()
print(bestConfig['values'])

model_best.evaluate(X_test,y_test)

save model for year: 2009
{'Nodes_choice': 3, 'activation': 'tanh', 'activate_dense_last': 'sigmoid', 'lr': 0.008294903912014776}
2466/2466 [==============================] - 6s 3ms/step - loss: 8.8885e-04 - R2: 0.8732 - mae: 0.0219


[0.0008888478623703122, 0.8732348084449768, 0.021877549588680267]

### 2010

In [48]:
from sklearn.preprocessing import StandardScaler
year = 2010
data_train = pd.read_csv(os.path.join('D:/SGYL/SM_results_data/Train/split/','train_data_'+str(year)+'.csv'))
data_validate = pd.read_csv(os.path.join('D:/SGYL/SM_results_data/Train/split/','validate_data_'+str(year)+'.csv'))
data_test = pd.read_csv(os.path.join('D:/SGYL/SM_results_data/Train/split/','test_data_'+str(year)+'.csv'))

X_train = data_train.drop(['SM'],axis = 1)
y_train = data_train['SM'].copy()   
X_validate = data_validate.drop(['SM'],axis = 1)
y_validate = data_validate['SM'].copy()
X_test = data_test.drop(['SM'],axis = 1)
y_test = data_test['SM'].copy()

standarder = StandardScaler()
X_train = standarder.fit_transform(X_train)
X_validate = standarder.transform(X_validate)
X_test = standarder.transform(X_test)

X_train = X_train.reshape(X_train.shape[0],X_train.shape[1],1)
X_validate = X_validate.reshape(X_validate.shape[0],X_validate.shape[1],1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1],1)

print('X_train shape:',X_train.shape)
print('X_validate shape:',X_validate.shape)

X_train shape: (650862, 19, 1)
X_validate shape: (195259, 19, 1)


In [49]:

def scheduler(epoch,lr):
    # 每隔30个epoch，学习率减小为原来的1/10
    if epoch % 30 == 0 and epoch != 0:
        print("lr changed to {}".format(lr * 0.1))
        return lr * 0.1
    else :
        return lr

callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=20,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
hypermodel = MyHyperModel()
tuner = keras_tuner.BayesianOptimization(
            hypermodel,
            objective=keras_tuner.Objective('val_R2', direction="max"),
            num_initial_points=50,
            max_trials=max_trials,
            overwrite = True,
            directory='D:/SGYL/SM_results_data/Bayesian_Opt/Full_ResNet/',
            project_name=('Full_ResNet_'+str(year)))
tuner.search(X_train,y_train,epochs=100,batch_size = 4096,validation_data=(X_validate,y_validate),shuffle = True,callbacks = [reduce_lr,callback_early_stopping],verbose = 2)

Trial 50 Complete [00h 03m 03s]
val_R2: 0.8341154456138611

Best val_R2 So Far: 0.8833414912223816
Total elapsed time: 02h 38m 25s
INFO:tensorflow:Oracle triggered exit


In [50]:
filepath="Full_ResNet_"+str(year)+"_best.hdf5"
callback_checkpoints = keras.callbacks.ModelCheckpoint(os.path.join('D:/SGYL/SM_results_data/check_points/Full_ResNet/',filepath),monitor='val_loss',save_best_only=True,verbose=1)
callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=50,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
callbacks = [reduce_lr,callback_early_stopping,callback_checkpoints]

if training_mode == 'model':
    model_best = tuner.get_best_models()[0]
else :
    model_best = tuner.hypermodel.build(tuner.get_best_hyperparameters()[0])
model_best.fit(X_train,y_train,epochs=300,batch_size = 4096,validation_data=(X_validate,y_validate),shuffle = True,callbacks = callbacks,verbose = 2)

Epoch 1/300

Epoch 00001: val_loss improved from inf to 0.00091, saving model to D:/SGYL/SM_results_data/check_points/Full_ResNet\Full_ResNet_2010_best.hdf5
159/159 - 2s - loss: 7.8484e-04 - R2: 0.8992 - mae: 0.0207 - val_loss: 9.0948e-04 - val_R2: 0.8832 - val_mae: 0.0220
Epoch 2/300

Epoch 00002: val_loss did not improve from 0.00091
159/159 - 2s - loss: 7.8564e-04 - R2: 0.8991 - mae: 0.0207 - val_loss: 9.1036e-04 - val_R2: 0.8831 - val_mae: 0.0220
Epoch 3/300

Epoch 00003: val_loss improved from 0.00091 to 0.00091, saving model to D:/SGYL/SM_results_data/check_points/Full_ResNet\Full_ResNet_2010_best.hdf5
159/159 - 2s - loss: 7.8347e-04 - R2: 0.8994 - mae: 0.0207 - val_loss: 9.0845e-04 - val_R2: 0.8834 - val_mae: 0.0219
Epoch 4/300

Epoch 00004: val_loss did not improve from 0.00091
159/159 - 2s - loss: 7.8534e-04 - R2: 0.8991 - mae: 0.0207 - val_loss: 9.1104e-04 - val_R2: 0.8830 - val_mae: 0.0220
Epoch 5/300

Epoch 00005: val_loss did not improve from 0.00091
159/159 - 2s - loss: 7

In [51]:
#save model
save_path = os.path.join('D:/SGYL/SM_results_data/model/Full_ResNet/','Full_ResNet_'+str(year)+'.hdf5')
model_best.save(save_path)
if (os.path.exists(save_path)):
    print('save model for year:',year)

#save model config
import json
bestConfig=tuner.get_best_hyperparameters()[0].get_config()
f = open(os.path.join('D:/SGYL/SM_results_data/Bayesian_Opt/Full_ResNet/Best_Config/','Full_ResNet_'+str(year)+'.json'), 'w')
json.dump(bestConfig,f)
f.close()
print(bestConfig['values'])

model_best.evaluate(X_test,y_test)

save model for year: 2010
{'Nodes_choice': 7, 'activation': 'tanh', 'activate_dense_last': 'sigmoid', 'lr': 0.004561896914979461}
2616/2616 [==============================] - 7s 3ms/step - loss: 9.0294e-04 - R2: 0.8748 - mae: 0.0221


[0.0009029418579302728, 0.8747758269309998, 0.022088300436735153]

### 2011

In [52]:
from sklearn.preprocessing import StandardScaler
year = 2011
data_train = pd.read_csv(os.path.join('D:/SGYL/SM_results_data/Train/split/','train_data_'+str(year)+'.csv'))
data_validate = pd.read_csv(os.path.join('D:/SGYL/SM_results_data/Train/split/','validate_data_'+str(year)+'.csv'))
data_test = pd.read_csv(os.path.join('D:/SGYL/SM_results_data/Train/split/','test_data_'+str(year)+'.csv'))

X_train = data_train.drop(['SM'],axis = 1)
y_train = data_train['SM'].copy()   
X_validate = data_validate.drop(['SM'],axis = 1)
y_validate = data_validate['SM'].copy()
X_test = data_test.drop(['SM'],axis = 1)
y_test = data_test['SM'].copy()

standarder = StandardScaler()
X_train = standarder.fit_transform(X_train)
X_validate = standarder.transform(X_validate)
X_test = standarder.transform(X_test)

X_train = X_train.reshape(X_train.shape[0],X_train.shape[1],1)
X_validate = X_validate.reshape(X_validate.shape[0],X_validate.shape[1],1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1],1)

print('X_train shape:',X_train.shape)
print('X_validate shape:',X_validate.shape)

X_train shape: (657288, 19, 1)
X_validate shape: (197187, 19, 1)


In [53]:

def scheduler(epoch,lr):
    # 每隔30个epoch，学习率减小为原来的1/10
    if epoch % 30 == 0 and epoch != 0:
        print("lr changed to {}".format(lr * 0.1))
        return lr * 0.1
    else :
        return lr

callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=20,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
hypermodel = MyHyperModel()
tuner = keras_tuner.BayesianOptimization(
            hypermodel,
            objective=keras_tuner.Objective('val_R2', direction="max"),
            num_initial_points=50,
            max_trials=max_trials,
            overwrite = True,
            directory='D:/SGYL/SM_results_data/Bayesian_Opt/Full_ResNet/',
            project_name=('Full_ResNet_'+str(year)))
tuner.search(X_train,y_train,epochs=100,batch_size = 4096,validation_data=(X_validate,y_validate),shuffle = True,callbacks = [reduce_lr,callback_early_stopping],verbose = 2)

Trial 50 Complete [00h 04m 01s]
val_R2: 0.8333718180656433

Best val_R2 So Far: 0.8886278867721558
Total elapsed time: 02h 41m 10s
INFO:tensorflow:Oracle triggered exit


In [54]:
filepath="Full_ResNet_"+str(year)+"_best.hdf5"
callback_checkpoints = keras.callbacks.ModelCheckpoint(os.path.join('D:/SGYL/SM_results_data/check_points/Full_ResNet/',filepath),monitor='val_loss',save_best_only=True,verbose=1)
callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=50,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
callbacks = [reduce_lr,callback_early_stopping,callback_checkpoints]

if training_mode == 'model':
    model_best = tuner.get_best_models()[0]
else :
    model_best = tuner.hypermodel.build(tuner.get_best_hyperparameters()[0])
model_best.fit(X_train,y_train,epochs=300,batch_size = 4096,validation_data=(X_validate,y_validate),shuffle = True,callbacks = callbacks,verbose = 2)

Epoch 1/300

Epoch 00001: val_loss improved from inf to 0.00081, saving model to D:/SGYL/SM_results_data/check_points/Full_ResNet\Full_ResNet_2011_best.hdf5
161/161 - 2s - loss: 7.1790e-04 - R2: 0.9012 - mae: 0.0198 - val_loss: 8.1143e-04 - val_R2: 0.8883 - val_mae: 0.0209
Epoch 2/300

Epoch 00002: val_loss improved from 0.00081 to 0.00081, saving model to D:/SGYL/SM_results_data/check_points/Full_ResNet\Full_ResNet_2011_best.hdf5
161/161 - 2s - loss: 7.1421e-04 - R2: 0.9017 - mae: 0.0197 - val_loss: 8.1056e-04 - val_R2: 0.8884 - val_mae: 0.0209
Epoch 3/300

Epoch 00003: val_loss improved from 0.00081 to 0.00081, saving model to D:/SGYL/SM_results_data/check_points/Full_ResNet\Full_ResNet_2011_best.hdf5
161/161 - 2s - loss: 7.1718e-04 - R2: 0.9013 - mae: 0.0197 - val_loss: 8.1044e-04 - val_R2: 0.8885 - val_mae: 0.0208
Epoch 4/300

Epoch 00004: val_loss did not improve from 0.00081
161/161 - 2s - loss: 7.1586e-04 - R2: 0.9015 - mae: 0.0197 - val_loss: 8.1065e-04 - val_R2: 0.8884 - val_m

In [55]:
#save model
save_path = os.path.join('D:/SGYL/SM_results_data/model/Full_ResNet/','Full_ResNet_'+str(year)+'.hdf5')
model_best.save(save_path)
if (os.path.exists(save_path)):
    print('save model for year:',year)

#save model config
import json
bestConfig=tuner.get_best_hyperparameters()[0].get_config()
f = open(os.path.join('D:/SGYL/SM_results_data/Bayesian_Opt/Full_ResNet/Best_Config/','Full_ResNet_'+str(year)+'.json'), 'w')
json.dump(bestConfig,f)
f.close()
print(bestConfig['values'])

model_best.evaluate(X_test,y_test)

save model for year: 2011
{'Nodes_choice': 3, 'activation': 'tanh', 'activate_dense_last': 'sigmoid', 'lr': 0.008294903912014776}
2641/2641 [==============================] - 7s 2ms/step - loss: 8.1258e-04 - R2: 0.8787 - mae: 0.0209


[0.0008125774911604822, 0.8787070512771606, 0.02094593271613121]

### 2012 

In [56]:
from sklearn.preprocessing import StandardScaler
year = 2012
data_train = pd.read_csv(os.path.join('D:/SGYL/SM_results_data/Train/split/','train_data_'+str(year)+'.csv'))
data_validate = pd.read_csv(os.path.join('D:/SGYL/SM_results_data/Train/split/','validate_data_'+str(year)+'.csv'))
data_test = pd.read_csv(os.path.join('D:/SGYL/SM_results_data/Train/split/','test_data_'+str(year)+'.csv'))

X_train = data_train.drop(['SM'],axis = 1)
y_train = data_train['SM'].copy()   
X_validate = data_validate.drop(['SM'],axis = 1)
y_validate = data_validate['SM'].copy()
X_test = data_test.drop(['SM'],axis = 1)
y_test = data_test['SM'].copy()

standarder = StandardScaler()
X_train = standarder.fit_transform(X_train)
X_validate = standarder.transform(X_validate)
X_test = standarder.transform(X_test)

X_train = X_train.reshape(X_train.shape[0],X_train.shape[1],1)
X_validate = X_validate.reshape(X_validate.shape[0],X_validate.shape[1],1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1],1)

print('X_train shape:',X_train.shape)
print('X_validate shape:',X_validate.shape)

X_train shape: (660527, 19, 1)
X_validate shape: (198158, 19, 1)


In [57]:

def scheduler(epoch,lr):
    # 每隔30个epoch，学习率减小为原来的1/10
    if epoch % 30 == 0 and epoch != 0:
        print("lr changed to {}".format(lr * 0.1))
        return lr * 0.1
    else :
        return lr

callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=20,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
hypermodel = MyHyperModel()
tuner = keras_tuner.BayesianOptimization(
            hypermodel,
            objective=keras_tuner.Objective('val_R2', direction="max"),
            num_initial_points=50,
            max_trials=max_trials,
            overwrite = True,
            directory='D:/SGYL/SM_results_data/Bayesian_Opt/Full_ResNet/',
            project_name=('Full_ResNet_'+str(year)))
tuner.search(X_train,y_train,epochs=100,batch_size = 4096,validation_data=(X_validate,y_validate),shuffle = True,callbacks = [reduce_lr,callback_early_stopping],verbose = 2)

Trial 50 Complete [00h 03m 24s]
val_R2: 0.8547228574752808

Best val_R2 So Far: 0.8994717001914978
Total elapsed time: 02h 39m 21s
INFO:tensorflow:Oracle triggered exit


In [58]:
filepath="Full_ResNet_"+str(year)+"_best.hdf5"
callback_checkpoints = keras.callbacks.ModelCheckpoint(os.path.join('D:/SGYL/SM_results_data/check_points/Full_ResNet/',filepath),monitor='val_loss',save_best_only=True,verbose=1)
callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=50,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
callbacks = [reduce_lr,callback_early_stopping,callback_checkpoints]

if training_mode == 'model':
    model_best = tuner.get_best_models()[0]
else :
    model_best = tuner.hypermodel.build(tuner.get_best_hyperparameters()[0])
model_best.fit(X_train,y_train,epochs=300,batch_size = 4096,validation_data=(X_validate,y_validate),shuffle = True,callbacks = callbacks,verbose = 2)

Epoch 1/300

Epoch 00001: val_loss improved from inf to 0.00082, saving model to D:/SGYL/SM_results_data/check_points/Full_ResNet\Full_ResNet_2012_best.hdf5
162/162 - 2s - loss: 7.2138e-04 - R2: 0.9110 - mae: 0.0197 - val_loss: 8.1711e-04 - val_R2: 0.8992 - val_mae: 0.0207
Epoch 2/300

Epoch 00002: val_loss improved from 0.00082 to 0.00082, saving model to D:/SGYL/SM_results_data/check_points/Full_ResNet\Full_ResNet_2012_best.hdf5
162/162 - 2s - loss: 7.2020e-04 - R2: 0.9111 - mae: 0.0197 - val_loss: 8.1688e-04 - val_R2: 0.8992 - val_mae: 0.0207
Epoch 3/300

Epoch 00003: val_loss did not improve from 0.00082
162/162 - 2s - loss: 7.1740e-04 - R2: 0.9115 - mae: 0.0197 - val_loss: 8.1929e-04 - val_R2: 0.8989 - val_mae: 0.0207
Epoch 4/300

Epoch 00004: val_loss improved from 0.00082 to 0.00082, saving model to D:/SGYL/SM_results_data/check_points/Full_ResNet\Full_ResNet_2012_best.hdf5
162/162 - 2s - loss: 7.1861e-04 - R2: 0.9114 - mae: 0.0197 - val_loss: 8.1649e-04 - val_R2: 0.8992 - val_m

In [59]:
#save model
save_path = os.path.join('D:/SGYL/SM_results_data/model/Full_ResNet/','Full_ResNet_'+str(year)+'.hdf5')
model_best.save(save_path)
if (os.path.exists(save_path)):
    print('save model for year:',year)

#save model config
import json
bestConfig=tuner.get_best_hyperparameters()[0].get_config()
f = open(os.path.join('D:/SGYL/SM_results_data/Bayesian_Opt/Full_ResNet/Best_Config/','Full_ResNet_'+str(year)+'.json'), 'w')
json.dump(bestConfig,f)
f.close()
print(bestConfig['values'])

model_best.evaluate(X_test,y_test)

save model for year: 2012
{'Nodes_choice': 3, 'activation': 'tanh', 'activate_dense_last': 'sigmoid', 'lr': 0.006882586554868881}
2654/2654 [==============================] - 7s 2ms/step - loss: 8.1478e-04 - R2: 0.8942 - mae: 0.0207


[0.000814782571978867, 0.8942148089408875, 0.020677074790000916]

### 2013

In [60]:
from sklearn.preprocessing import StandardScaler
year = 2013
data_train = pd.read_csv(os.path.join('D:/SGYL/SM_results_data/Train/split/','train_data_'+str(year)+'.csv'))
data_validate = pd.read_csv(os.path.join('D:/SGYL/SM_results_data/Train/split/','validate_data_'+str(year)+'.csv'))
data_test = pd.read_csv(os.path.join('D:/SGYL/SM_results_data/Train/split/','test_data_'+str(year)+'.csv'))

X_train = data_train.drop(['SM'],axis = 1)
y_train = data_train['SM'].copy()   
X_validate = data_validate.drop(['SM'],axis = 1)
y_validate = data_validate['SM'].copy()
X_test = data_test.drop(['SM'],axis = 1)
y_test = data_test['SM'].copy()

standarder = StandardScaler()
X_train = standarder.fit_transform(X_train)
X_validate = standarder.transform(X_validate)
X_test = standarder.transform(X_test)

X_train = X_train.reshape(X_train.shape[0],X_train.shape[1],1)
X_validate = X_validate.reshape(X_validate.shape[0],X_validate.shape[1],1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1],1)

print('X_train shape:',X_train.shape)
print('X_validate shape:',X_validate.shape)

X_train shape: (647136, 19, 1)
X_validate shape: (194140, 19, 1)


In [61]:

def scheduler(epoch,lr):
    # 每隔30个epoch，学习率减小为原来的1/10
    if epoch % 30 == 0 and epoch != 0:
        print("lr changed to {}".format(lr * 0.1))
        return lr * 0.1
    else :
        return lr

callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=20,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
hypermodel = MyHyperModel()
tuner = keras_tuner.BayesianOptimization(
            hypermodel,
            objective=keras_tuner.Objective('val_R2', direction="max"),
            num_initial_points=50,
            max_trials=max_trials,
            overwrite = True,
            directory='D:/SGYL/SM_results_data/Bayesian_Opt/Full_ResNet/',
            project_name=('Full_ResNet_'+str(year)))
tuner.search(X_train,y_train,epochs=100,batch_size = 4096,validation_data=(X_validate,y_validate),shuffle = True,callbacks = [reduce_lr,callback_early_stopping],verbose = 2)

Trial 50 Complete [00h 02m 54s]
val_R2: 0.7313196063041687

Best val_R2 So Far: 0.8951126933097839
Total elapsed time: 02h 41m 19s
INFO:tensorflow:Oracle triggered exit


In [62]:
filepath="Full_ResNet_"+str(year)+"_best.hdf5"
callback_checkpoints = keras.callbacks.ModelCheckpoint(os.path.join('D:/SGYL/SM_results_data/check_points/Full_ResNet/',filepath),monitor='val_loss',save_best_only=True,verbose=1)
callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=50,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
callbacks = [reduce_lr,callback_early_stopping,callback_checkpoints]

if training_mode == 'model':
    model_best = tuner.get_best_models()[0]
else :
    model_best = tuner.hypermodel.build(tuner.get_best_hyperparameters()[0])
model_best.fit(X_train,y_train,epochs=300,batch_size = 4096,validation_data=(X_validate,y_validate),shuffle = True,callbacks = callbacks,verbose = 2)

Epoch 1/300

Epoch 00001: val_loss improved from inf to 0.00082, saving model to D:/SGYL/SM_results_data/check_points/Full_ResNet\Full_ResNet_2013_best.hdf5
158/158 - 2s - loss: 7.4801e-04 - R2: 0.9035 - mae: 0.0202 - val_loss: 8.1852e-04 - val_R2: 0.8946 - val_mae: 0.0209
Epoch 2/300

Epoch 00002: val_loss improved from 0.00082 to 0.00082, saving model to D:/SGYL/SM_results_data/check_points/Full_ResNet\Full_ResNet_2013_best.hdf5
158/158 - 2s - loss: 7.4727e-04 - R2: 0.9036 - mae: 0.0202 - val_loss: 8.1517e-04 - val_R2: 0.8951 - val_mae: 0.0209
Epoch 3/300

Epoch 00003: val_loss did not improve from 0.00082
158/158 - 2s - loss: 7.4644e-04 - R2: 0.9037 - mae: 0.0201 - val_loss: 8.1662e-04 - val_R2: 0.8949 - val_mae: 0.0209
Epoch 4/300

Epoch 00004: val_loss did not improve from 0.00082
158/158 - 2s - loss: 7.4868e-04 - R2: 0.9034 - mae: 0.0202 - val_loss: 8.1561e-04 - val_R2: 0.8950 - val_mae: 0.0209
Epoch 5/300

Epoch 00005: val_loss did not improve from 0.00082
158/158 - 2s - loss: 7

In [63]:
#save model
save_path = os.path.join('D:/SGYL/SM_results_data/model/Full_ResNet/','Full_ResNet_'+str(year)+'.hdf5')
model_best.save(save_path)
if (os.path.exists(save_path)):
    print('save model for year:',year)

#save model config
import json
bestConfig=tuner.get_best_hyperparameters()[0].get_config()
f = open(os.path.join('D:/SGYL/SM_results_data/Bayesian_Opt/Full_ResNet/Best_Config/','Full_ResNet_'+str(year)+'.json'), 'w')
json.dump(bestConfig,f)
f.close()
print(bestConfig['values'])

model_best.evaluate(X_test,y_test)

save model for year: 2013
{'Nodes_choice': 6, 'activation': 'tanh', 'activate_dense_last': 'sigmoid', 'lr': 0.006610896434199781}
2601/2601 [==============================] - 8s 3ms/step - loss: 8.1257e-04 - R2: 0.8876 - mae: 0.0208


[0.0008125724270939827, 0.8875662684440613, 0.020767999812960625]

### 2014

In [64]:
from sklearn.preprocessing import StandardScaler
year = 2014
data_train = pd.read_csv(os.path.join('D:/SGYL/SM_results_data/Train/split/','train_data_'+str(year)+'.csv'))
data_validate = pd.read_csv(os.path.join('D:/SGYL/SM_results_data/Train/split/','validate_data_'+str(year)+'.csv'))
data_test = pd.read_csv(os.path.join('D:/SGYL/SM_results_data/Train/split/','test_data_'+str(year)+'.csv'))

X_train = data_train.drop(['SM'],axis = 1)
y_train = data_train['SM'].copy()   
X_validate = data_validate.drop(['SM'],axis = 1)
y_validate = data_validate['SM'].copy()
X_test = data_test.drop(['SM'],axis = 1)
y_test = data_test['SM'].copy()

standarder = StandardScaler()
X_train = standarder.fit_transform(X_train)
X_validate = standarder.transform(X_validate)
X_test = standarder.transform(X_test)

X_train = X_train.reshape(X_train.shape[0],X_train.shape[1],1)
X_validate = X_validate.reshape(X_validate.shape[0],X_validate.shape[1],1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1],1)

print('X_train shape:',X_train.shape)
print('X_validate shape:',X_validate.shape)

X_train shape: (603515, 19, 1)
X_validate shape: (181055, 19, 1)


In [65]:

def scheduler(epoch,lr):
    # 每隔30个epoch，学习率减小为原来的1/10
    if epoch % 30 == 0 and epoch != 0:
        print("lr changed to {}".format(lr * 0.1))
        return lr * 0.1
    else :
        return lr

callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=20,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
hypermodel = MyHyperModel()
tuner = keras_tuner.BayesianOptimization(
            hypermodel,
            objective=keras_tuner.Objective('val_R2', direction="max"),
            num_initial_points=50,
            max_trials=max_trials,
            overwrite = True,
            directory='D:/SGYL/SM_results_data/Bayesian_Opt/Full_ResNet/',
            project_name=('Full_ResNet_'+str(year)))
tuner.search(X_train,y_train,epochs=100,batch_size = 4096,validation_data=(X_validate,y_validate),shuffle = True,callbacks = [reduce_lr,callback_early_stopping],verbose = 2)

Trial 50 Complete [00h 03m 26s]
val_R2: 0.8683093786239624

Best val_R2 So Far: 0.9086082577705383
Total elapsed time: 02h 31m 21s
INFO:tensorflow:Oracle triggered exit


In [66]:
filepath="Full_ResNet_"+str(year)+"_best.hdf5"
callback_checkpoints = keras.callbacks.ModelCheckpoint(os.path.join('D:/SGYL/SM_results_data/check_points/Full_ResNet/',filepath),monitor='val_loss',save_best_only=True,verbose=1)
callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=50,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
callbacks = [reduce_lr,callback_early_stopping,callback_checkpoints]

if training_mode == 'model':
    model_best = tuner.get_best_models()[0]
else :
    model_best = tuner.hypermodel.build(tuner.get_best_hyperparameters()[0])
model_best.fit(X_train,y_train,epochs=300,batch_size = 4096,validation_data=(X_validate,y_validate),shuffle = True,callbacks = callbacks,verbose = 2)

Epoch 1/300

Epoch 00001: val_loss improved from inf to 0.00076, saving model to D:/SGYL/SM_results_data/check_points/Full_ResNet\Full_ResNet_2014_best.hdf5
148/148 - 2s - loss: 6.6024e-04 - R2: 0.9204 - mae: 0.0189 - val_loss: 7.5978e-04 - val_R2: 0.9087 - val_mae: 0.0201
Epoch 2/300

Epoch 00002: val_loss did not improve from 0.00076
148/148 - 2s - loss: 6.5892e-04 - R2: 0.9205 - mae: 0.0189 - val_loss: 7.6223e-04 - val_R2: 0.9084 - val_mae: 0.0201
Epoch 3/300

Epoch 00003: val_loss did not improve from 0.00076
148/148 - 2s - loss: 6.6335e-04 - R2: 0.9200 - mae: 0.0190 - val_loss: 7.6441e-04 - val_R2: 0.9081 - val_mae: 0.0202
Epoch 4/300

Epoch 00004: val_loss did not improve from 0.00076
148/148 - 2s - loss: 6.5983e-04 - R2: 0.9204 - mae: 0.0189 - val_loss: 7.6160e-04 - val_R2: 0.9085 - val_mae: 0.0201
Epoch 5/300

Epoch 00005: val_loss improved from 0.00076 to 0.00076, saving model to D:/SGYL/SM_results_data/check_points/Full_ResNet\Full_ResNet_2014_best.hdf5
148/148 - 2s - loss: 6

In [67]:
#save model
save_path = os.path.join('D:/SGYL/SM_results_data/model/Full_ResNet/','Full_ResNet_'+str(year)+'.hdf5')
model_best.save(save_path)
if (os.path.exists(save_path)):
    print('save model for year:',year)

#save model config
import json
bestConfig=tuner.get_best_hyperparameters()[0].get_config()
f = open(os.path.join('D:/SGYL/SM_results_data/Bayesian_Opt/Full_ResNet/Best_Config/','Full_ResNet_'+str(year)+'.json'), 'w')
json.dump(bestConfig,f)
f.close()
print(bestConfig['values'])

model_best.evaluate(X_test,y_test)

save model for year: 2014
{'Nodes_choice': 3, 'activation': 'tanh', 'activate_dense_last': 'sigmoid', 'lr': 0.005347081268660574}
2425/2425 [==============================] - 6s 3ms/step - loss: 7.5498e-04 - R2: 0.9019 - mae: 0.0201


[0.0007549847941845655, 0.9018838405609131, 0.0200990978628397]

### 2015

In [68]:
from sklearn.preprocessing import StandardScaler
year = 2015
data_train = pd.read_csv(os.path.join('D:/SGYL/SM_results_data/Train/split/','train_data_'+str(year)+'.csv'))
data_validate = pd.read_csv(os.path.join('D:/SGYL/SM_results_data/Train/split/','validate_data_'+str(year)+'.csv'))
data_test = pd.read_csv(os.path.join('D:/SGYL/SM_results_data/Train/split/','test_data_'+str(year)+'.csv'))

X_train = data_train.drop(['SM'],axis = 1)
y_train = data_train['SM'].copy()   
X_validate = data_validate.drop(['SM'],axis = 1)
y_validate = data_validate['SM'].copy()
X_test = data_test.drop(['SM'],axis = 1)
y_test = data_test['SM'].copy()

standarder = StandardScaler()
X_train = standarder.fit_transform(X_train)
X_validate = standarder.transform(X_validate)
X_test = standarder.transform(X_test)

X_train = X_train.reshape(X_train.shape[0],X_train.shape[1],1)
X_validate = X_validate.reshape(X_validate.shape[0],X_validate.shape[1],1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1],1)

print('X_train shape:',X_train.shape)
print('X_validate shape:',X_validate.shape)

X_train shape: (606046, 19, 1)
X_validate shape: (181813, 19, 1)


In [69]:

def scheduler(epoch,lr):
    # 每隔30个epoch，学习率减小为原来的1/10
    if epoch % 30 == 0 and epoch != 0:
        print("lr changed to {}".format(lr * 0.1))
        return lr * 0.1
    else :
        return lr

callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=20,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
hypermodel = MyHyperModel()
tuner = keras_tuner.BayesianOptimization(
            hypermodel,
            objective=keras_tuner.Objective('val_R2', direction="max"),
            num_initial_points=50,
            max_trials=max_trials,
            overwrite = True,
            directory='D:/SGYL/SM_results_data/Bayesian_Opt/Full_ResNet/',
            project_name=('Full_ResNet_'+str(year)))
tuner.search(X_train,y_train,epochs=100,batch_size = 4096,validation_data=(X_validate,y_validate),shuffle = True,callbacks = [reduce_lr,callback_early_stopping],verbose = 2)

Trial 50 Complete [00h 02m 52s]
val_R2: 0.8371343016624451

Best val_R2 So Far: 0.8960535526275635
Total elapsed time: 02h 33m 02s
INFO:tensorflow:Oracle triggered exit


In [70]:
filepath="Full_ResNet_"+str(year)+"_best.hdf5"
callback_checkpoints = keras.callbacks.ModelCheckpoint(os.path.join('D:/SGYL/SM_results_data/check_points/Full_ResNet/',filepath),monitor='val_loss',save_best_only=True,verbose=1)
callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=50,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
callbacks = [reduce_lr,callback_early_stopping,callback_checkpoints]

if training_mode == 'model':
    model_best = tuner.get_best_models()[0]
else :
    model_best = tuner.hypermodel.build(tuner.get_best_hyperparameters()[0])
model_best.fit(X_train,y_train,epochs=300,batch_size = 4096,validation_data=(X_validate,y_validate),shuffle = True,callbacks = callbacks,verbose = 2)

Epoch 1/300

Epoch 00001: val_loss improved from inf to 0.00084, saving model to D:/SGYL/SM_results_data/check_points/Full_ResNet\Full_ResNet_2015_best.hdf5
148/148 - 2s - loss: 7.6174e-04 - R2: 0.9058 - mae: 0.0204 - val_loss: 8.3845e-04 - val_R2: 0.8960 - val_mae: 0.0213
Epoch 2/300

Epoch 00002: val_loss did not improve from 0.00084
148/148 - 2s - loss: 7.5794e-04 - R2: 0.9062 - mae: 0.0204 - val_loss: 8.3887e-04 - val_R2: 0.8960 - val_mae: 0.0213
Epoch 3/300

Epoch 00003: val_loss improved from 0.00084 to 0.00084, saving model to D:/SGYL/SM_results_data/check_points/Full_ResNet\Full_ResNet_2015_best.hdf5
148/148 - 2s - loss: 7.5928e-04 - R2: 0.9061 - mae: 0.0204 - val_loss: 8.3838e-04 - val_R2: 0.8960 - val_mae: 0.0213
Epoch 4/300

Epoch 00004: val_loss did not improve from 0.00084
148/148 - 2s - loss: 7.5908e-04 - R2: 0.9061 - mae: 0.0204 - val_loss: 8.3903e-04 - val_R2: 0.8959 - val_mae: 0.0213
Epoch 5/300

Epoch 00005: val_loss did not improve from 0.00084
148/148 - 2s - loss: 7

In [71]:
#save model
save_path = os.path.join('D:/SGYL/SM_results_data/model/Full_ResNet/','Full_ResNet_'+str(year)+'.hdf5')
model_best.save(save_path)
if (os.path.exists(save_path)):
    print('save model for year:',year)

#save model config
import json
bestConfig=tuner.get_best_hyperparameters()[0].get_config()
f = open(os.path.join('D:/SGYL/SM_results_data/Bayesian_Opt/Full_ResNet/Best_Config/','Full_ResNet_'+str(year)+'.json'), 'w')
json.dump(bestConfig,f)
f.close()
print(bestConfig['values'])

model_best.evaluate(X_test,y_test)

save model for year: 2015
{'Nodes_choice': 7, 'activation': 'tanh', 'activate_dense_last': 'sigmoid', 'lr': 0.0035072152287283025}
2436/2436 [==============================] - 7s 3ms/step - loss: 8.4508e-04 - R2: -inf - mae: 0.0213


[0.0008450773893855512, -inf, 0.021254772320389748]

### 2016

In [72]:
from sklearn.preprocessing import StandardScaler
year = 2016
data_train = pd.read_csv(os.path.join('D:/SGYL/SM_results_data/Train/split/','train_data_'+str(year)+'.csv'))
data_validate = pd.read_csv(os.path.join('D:/SGYL/SM_results_data/Train/split/','validate_data_'+str(year)+'.csv'))
data_test = pd.read_csv(os.path.join('D:/SGYL/SM_results_data/Train/split/','test_data_'+str(year)+'.csv'))

X_train = data_train.drop(['SM'],axis = 1)
y_train = data_train['SM'].copy()   
X_validate = data_validate.drop(['SM'],axis = 1)
y_validate = data_validate['SM'].copy()
X_test = data_test.drop(['SM'],axis = 1)
y_test = data_test['SM'].copy()

standarder = StandardScaler()
X_train = standarder.fit_transform(X_train)
X_validate = standarder.transform(X_validate)
X_test = standarder.transform(X_test)

X_train = X_train.reshape(X_train.shape[0],X_train.shape[1],1)
X_validate = X_validate.reshape(X_validate.shape[0],X_validate.shape[1],1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1],1)

print('X_train shape:',X_train.shape)
print('X_validate shape:',X_validate.shape)

X_train shape: (636232, 19, 1)
X_validate shape: (190870, 19, 1)


In [73]:

def scheduler(epoch,lr):
    # 每隔30个epoch，学习率减小为原来的1/10
    if epoch % 30 == 0 and epoch != 0:
        print("lr changed to {}".format(lr * 0.1))
        return lr * 0.1
    else :
        return lr

callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=20,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
hypermodel = MyHyperModel()
tuner = keras_tuner.BayesianOptimization(
            hypermodel,
            objective=keras_tuner.Objective('val_R2', direction="max"),
            num_initial_points=50,
            max_trials=max_trials,
            overwrite = True,
            directory='D:/SGYL/SM_results_data/Bayesian_Opt/Full_ResNet/',
            project_name=('Full_ResNet_'+str(year)))
tuner.search(X_train,y_train,epochs=100,batch_size = 4096,validation_data=(X_validate,y_validate),shuffle = True,callbacks = [reduce_lr,callback_early_stopping],verbose = 2)

Trial 50 Complete [00h 03m 11s]
val_R2: 0.8980234265327454

Best val_R2 So Far: 0.9104040265083313
Total elapsed time: 02h 39m 46s
INFO:tensorflow:Oracle triggered exit


In [74]:
filepath="Full_ResNet_"+str(year)+"_best.hdf5"
callback_checkpoints = keras.callbacks.ModelCheckpoint(os.path.join('D:/SGYL/SM_results_data/check_points/Full_ResNet/',filepath),monitor='val_loss',save_best_only=True,verbose=1)
callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=50,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
callbacks = [reduce_lr,callback_early_stopping,callback_checkpoints]

if training_mode == 'model':
    model_best = tuner.get_best_models()[0]
else :
    model_best = tuner.hypermodel.build(tuner.get_best_hyperparameters()[0])
model_best.fit(X_train,y_train,epochs=300,batch_size = 4096,validation_data=(X_validate,y_validate),shuffle = True,callbacks = callbacks,verbose = 2)

Epoch 1/300

Epoch 00001: val_loss improved from inf to 0.00077, saving model to D:/SGYL/SM_results_data/check_points/Full_ResNet\Full_ResNet_2016_best.hdf5
156/156 - 2s - loss: 6.8456e-04 - R2: 0.9203 - mae: 0.0193 - val_loss: 7.6794e-04 - val_R2: 0.9103 - val_mae: 0.0203
Epoch 2/300

Epoch 00002: val_loss improved from 0.00077 to 0.00077, saving model to D:/SGYL/SM_results_data/check_points/Full_ResNet\Full_ResNet_2016_best.hdf5
156/156 - 2s - loss: 6.8553e-04 - R2: 0.9202 - mae: 0.0193 - val_loss: 7.6530e-04 - val_R2: 0.9106 - val_mae: 0.0203
Epoch 3/300

Epoch 00003: val_loss did not improve from 0.00077
156/156 - 2s - loss: 6.8508e-04 - R2: 0.9202 - mae: 0.0193 - val_loss: 7.6805e-04 - val_R2: 0.9103 - val_mae: 0.0203
Epoch 4/300

Epoch 00004: val_loss did not improve from 0.00077
156/156 - 2s - loss: 6.8528e-04 - R2: 0.9201 - mae: 0.0193 - val_loss: 7.6884e-04 - val_R2: 0.9102 - val_mae: 0.0203
Epoch 5/300

Epoch 00005: val_loss did not improve from 0.00077
156/156 - 2s - loss: 6

In [75]:
#save model
save_path = os.path.join('D:/SGYL/SM_results_data/model/Full_ResNet/','Full_ResNet_'+str(year)+'.hdf5')
model_best.save(save_path)
if (os.path.exists(save_path)):
    print('save model for year:',year)

#save model config
import json
bestConfig=tuner.get_best_hyperparameters()[0].get_config()
f = open(os.path.join('D:/SGYL/SM_results_data/Bayesian_Opt/Full_ResNet/Best_Config/','Full_ResNet_'+str(year)+'.json'), 'w')
json.dump(bestConfig,f)
f.close()
print(bestConfig['values'])

model_best.evaluate(X_test,y_test)

save model for year: 2016
{'Nodes_choice': 7, 'activation': 'tanh', 'activate_dense_last': 'sigmoid', 'lr': 0.00334117424389347}
2557/2557 [==============================] - 8s 3ms/step - loss: 7.4825e-04 - R2: 0.9060 - mae: 0.0201


[0.0007482543587684631, 0.905997097492218, 0.020064279437065125]

### 2017

In [76]:
from sklearn.preprocessing import StandardScaler
year = 2017
data_train = pd.read_csv(os.path.join('D:/SGYL/SM_results_data/Train/split/','train_data_'+str(year)+'.csv'))
data_validate = pd.read_csv(os.path.join('D:/SGYL/SM_results_data/Train/split/','validate_data_'+str(year)+'.csv'))
data_test = pd.read_csv(os.path.join('D:/SGYL/SM_results_data/Train/split/','test_data_'+str(year)+'.csv'))

X_train = data_train.drop(['SM'],axis = 1)
y_train = data_train['SM'].copy()   
X_validate = data_validate.drop(['SM'],axis = 1)
y_validate = data_validate['SM'].copy()
X_test = data_test.drop(['SM'],axis = 1)
y_test = data_test['SM'].copy()

standarder = StandardScaler()
X_train = standarder.fit_transform(X_train)
X_validate = standarder.transform(X_validate)
X_test = standarder.transform(X_test)

X_train = X_train.reshape(X_train.shape[0],X_train.shape[1],1)
X_validate = X_validate.reshape(X_validate.shape[0],X_validate.shape[1],1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1],1)

print('X_train shape:',X_train.shape)
print('X_validate shape:',X_validate.shape)

X_train shape: (637490, 19, 1)
X_validate shape: (191247, 19, 1)


In [77]:

def scheduler(epoch,lr):
    # 每隔30个epoch，学习率减小为原来的1/10
    if epoch % 30 == 0 and epoch != 0:
        print("lr changed to {}".format(lr * 0.1))
        return lr * 0.1
    else :
        return lr

callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=20,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
hypermodel = MyHyperModel()
tuner = keras_tuner.BayesianOptimization(
            hypermodel,
            objective=keras_tuner.Objective('val_R2', direction="max"),
            num_initial_points=50,
            max_trials=max_trials,
            overwrite = True,
            directory='D:/SGYL/SM_results_data/Bayesian_Opt/Full_ResNet/',
            project_name=('Full_ResNet_'+str(year)))
tuner.search(X_train,y_train,epochs=100,batch_size = 4096,validation_data=(X_validate,y_validate),shuffle = True,callbacks = [reduce_lr,callback_early_stopping],verbose = 2)

Trial 50 Complete [00h 03m 52s]
val_R2: 0.8414250016212463

Best val_R2 So Far: 0.907659649848938
Total elapsed time: 02h 40m 47s
INFO:tensorflow:Oracle triggered exit


In [78]:
filepath="Full_ResNet_"+str(year)+"_best.hdf5"
callback_checkpoints = keras.callbacks.ModelCheckpoint(os.path.join('D:/SGYL/SM_results_data/check_points/Full_ResNet/',filepath),monitor='val_loss',save_best_only=True,verbose=1)
callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=50,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
callbacks = [reduce_lr,callback_early_stopping,callback_checkpoints]

if training_mode == 'model':
    model_best = tuner.get_best_models()[0]
else :
    model_best = tuner.hypermodel.build(tuner.get_best_hyperparameters()[0])
model_best.fit(X_train,y_train,epochs=300,batch_size = 4096,validation_data=(X_validate,y_validate),shuffle = True,callbacks = callbacks,verbose = 2)

Epoch 1/300

Epoch 00001: val_loss improved from inf to 0.00080, saving model to D:/SGYL/SM_results_data/check_points/Full_ResNet\Full_ResNet_2017_best.hdf5
156/156 - 3s - loss: 7.3400e-04 - R2: 0.9149 - mae: 0.0200 - val_loss: 7.9839e-04 - val_R2: 0.9075 - val_mae: 0.0207
Epoch 2/300

Epoch 00002: val_loss improved from 0.00080 to 0.00080, saving model to D:/SGYL/SM_results_data/check_points/Full_ResNet\Full_ResNet_2017_best.hdf5
156/156 - 3s - loss: 7.3451e-04 - R2: 0.9148 - mae: 0.0201 - val_loss: 7.9837e-04 - val_R2: 0.9075 - val_mae: 0.0207
Epoch 3/300

Epoch 00003: val_loss improved from 0.00080 to 0.00080, saving model to D:/SGYL/SM_results_data/check_points/Full_ResNet\Full_ResNet_2017_best.hdf5
156/156 - 2s - loss: 7.3223e-04 - R2: 0.9151 - mae: 0.0200 - val_loss: 7.9633e-04 - val_R2: 0.9077 - val_mae: 0.0207
Epoch 4/300

Epoch 00004: val_loss did not improve from 0.00080
156/156 - 2s - loss: 7.3367e-04 - R2: 0.9149 - mae: 0.0200 - val_loss: 7.9735e-04 - val_R2: 0.9076 - val_m

In [79]:
#save model
save_path = os.path.join('D:/SGYL/SM_results_data/model/Full_ResNet/','Full_ResNet_'+str(year)+'.hdf5')
model_best.save(save_path)
if (os.path.exists(save_path)):
    print('save model for year:',year)

#save model config
import json
bestConfig=tuner.get_best_hyperparameters()[0].get_config()
f = open(os.path.join('D:/SGYL/SM_results_data/Bayesian_Opt/Full_ResNet/Best_Config/','Full_ResNet_'+str(year)+'.json'), 'w')
json.dump(bestConfig,f)
f.close()
print(bestConfig['values'])

model_best.evaluate(X_test,y_test)

save model for year: 2017
{'Nodes_choice': 6, 'activation': 'tanh', 'activate_dense_last': 'sigmoid', 'lr': 0.00727915929887585}
2562/2562 [==============================] - 9s 4ms/step - loss: 8.0332e-04 - R2: 0.8998 - mae: 0.0208


[0.0008033236372284591, 0.8997791409492493, 0.0208100788295269]

### 2018

In [80]:
from sklearn.preprocessing import StandardScaler
year = 2018
data_train = pd.read_csv(os.path.join('D:/SGYL/SM_results_data/Train/split/','train_data_'+str(year)+'.csv'))
data_validate = pd.read_csv(os.path.join('D:/SGYL/SM_results_data/Train/split/','validate_data_'+str(year)+'.csv'))
data_test = pd.read_csv(os.path.join('D:/SGYL/SM_results_data/Train/split/','test_data_'+str(year)+'.csv'))

X_train = data_train.drop(['SM'],axis = 1)
y_train = data_train['SM'].copy()   
X_validate = data_validate.drop(['SM'],axis = 1)
y_validate = data_validate['SM'].copy()
X_test = data_test.drop(['SM'],axis = 1)
y_test = data_test['SM'].copy()

standarder = StandardScaler()
X_train = standarder.fit_transform(X_train)
X_validate = standarder.transform(X_validate)
X_test = standarder.transform(X_test)

X_train = X_train.reshape(X_train.shape[0],X_train.shape[1],1)
X_validate = X_validate.reshape(X_validate.shape[0],X_validate.shape[1],1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1],1)

print('X_train shape:',X_train.shape)
print('X_validate shape:',X_validate.shape)

X_train shape: (614272, 19, 1)
X_validate shape: (184282, 19, 1)


In [81]:

def scheduler(epoch,lr):
    # 每隔30个epoch，学习率减小为原来的1/10
    if epoch % 30 == 0 and epoch != 0:
        print("lr changed to {}".format(lr * 0.1))
        return lr * 0.1
    else :
        return lr

callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=20,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
hypermodel = MyHyperModel()
tuner = keras_tuner.BayesianOptimization(
            hypermodel,
            objective=keras_tuner.Objective('val_R2', direction="max"),
            num_initial_points=50,
            max_trials=max_trials,
            overwrite = True,
            directory='D:/SGYL/SM_results_data/Bayesian_Opt/Full_ResNet/',
            project_name=('Full_ResNet_'+str(year)))
tuner.search(X_train,y_train,epochs=100,batch_size = 4096,validation_data=(X_validate,y_validate),shuffle = True,callbacks = [reduce_lr,callback_early_stopping],verbose = 2)

Trial 50 Complete [00h 02m 58s]
val_R2: 0.8841688632965088

Best val_R2 So Far: 0.9076123833656311
Total elapsed time: 02h 30m 26s


INFO:tensorflow:Oracle triggered exit


In [82]:
filepath="Full_ResNet_"+str(year)+"_best.hdf5"
callback_checkpoints = keras.callbacks.ModelCheckpoint(os.path.join('D:/SGYL/SM_results_data/check_points/Full_ResNet/',filepath),monitor='val_loss',save_best_only=True,verbose=1)
callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=50,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
callbacks = [reduce_lr,callback_early_stopping,callback_checkpoints]

if training_mode == 'model':
    model_best = tuner.get_best_models()[0]
else :
    model_best = tuner.hypermodel.build(tuner.get_best_hyperparameters()[0])
model_best.fit(X_train,y_train,epochs=300,batch_size = 4096,validation_data=(X_validate,y_validate),shuffle = True,callbacks = callbacks,verbose = 2)

Epoch 1/300

Epoch 00001: val_loss improved from inf to 0.00082, saving model to D:/SGYL/SM_results_data/check_points/Full_ResNet\Full_ResNet_2018_best.hdf5
150/150 - 2s - loss: 7.7533e-04 - R2: 0.9121 - mae: 0.0206 - val_loss: 8.1752e-04 - val_R2: 0.9073 - val_mae: 0.0211
Epoch 2/300

Epoch 00002: val_loss improved from 0.00082 to 0.00082, saving model to D:/SGYL/SM_results_data/check_points/Full_ResNet\Full_ResNet_2018_best.hdf5
150/150 - 2s - loss: 7.7676e-04 - R2: 0.9120 - mae: 0.0206 - val_loss: 8.1527e-04 - val_R2: 0.9076 - val_mae: 0.0210
Epoch 3/300

Epoch 00003: val_loss did not improve from 0.00082
150/150 - 2s - loss: 7.7462e-04 - R2: 0.9122 - mae: 0.0206 - val_loss: 8.1569e-04 - val_R2: 0.9075 - val_mae: 0.0210
Epoch 4/300

Epoch 00004: val_loss did not improve from 0.00082
150/150 - 2s - loss: 7.7581e-04 - R2: 0.9121 - mae: 0.0206 - val_loss: 8.1588e-04 - val_R2: 0.9075 - val_mae: 0.0210
Epoch 5/300

Epoch 00005: val_loss did not improve from 0.00082
150/150 - 2s - loss: 7

In [83]:
#save model
save_path = os.path.join('D:/SGYL/SM_results_data/model/Full_ResNet/','Full_ResNet_'+str(year)+'.hdf5')
model_best.save(save_path)
if (os.path.exists(save_path)):
    print('save model for year:',year)

#save model config
import json
bestConfig=tuner.get_best_hyperparameters()[0].get_config()
f = open(os.path.join('D:/SGYL/SM_results_data/Bayesian_Opt/Full_ResNet/Best_Config/','Full_ResNet_'+str(year)+'.json'), 'w')
json.dump(bestConfig,f)
f.close()
print(bestConfig['values'])

model_best.evaluate(X_test,y_test)

save model for year: 2018
{'Nodes_choice': 6, 'activation': 'tanh', 'activate_dense_last': 'sigmoid', 'lr': 0.0024217363790330513}
2469/2469 [==============================] - 7s 3ms/step - loss: 8.2127e-04 - R2: 0.8997 - mae: 0.0211


[0.0008212663815356791, 0.899714469909668, 0.021085098385810852]

### 2019

In [84]:
from sklearn.preprocessing import StandardScaler
year = 2019
data_train = pd.read_csv(os.path.join('D:/SGYL/SM_results_data/Train/split/','train_data_'+str(year)+'.csv'))
data_validate = pd.read_csv(os.path.join('D:/SGYL/SM_results_data/Train/split/','validate_data_'+str(year)+'.csv'))
data_test = pd.read_csv(os.path.join('D:/SGYL/SM_results_data/Train/split/','test_data_'+str(year)+'.csv'))

X_train = data_train.drop(['SM'],axis = 1)
y_train = data_train['SM'].copy()   
X_validate = data_validate.drop(['SM'],axis = 1)
y_validate = data_validate['SM'].copy()
X_test = data_test.drop(['SM'],axis = 1)
y_test = data_test['SM'].copy()

standarder = StandardScaler()
X_train = standarder.fit_transform(X_train)
X_validate = standarder.transform(X_validate)
X_test = standarder.transform(X_test)

X_train = X_train.reshape(X_train.shape[0],X_train.shape[1],1)
X_validate = X_validate.reshape(X_validate.shape[0],X_validate.shape[1],1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1],1)

print('X_train shape:',X_train.shape)
print('X_validate shape:',X_validate.shape)

X_train shape: (625580, 19, 1)
X_validate shape: (187674, 19, 1)


In [85]:

def scheduler(epoch,lr):
    # 每隔30个epoch，学习率减小为原来的1/10
    if epoch % 30 == 0 and epoch != 0:
        print("lr changed to {}".format(lr * 0.1))
        return lr * 0.1
    else :
        return lr

callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=20,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
hypermodel = MyHyperModel()
tuner = keras_tuner.BayesianOptimization(
            hypermodel,
            objective=keras_tuner.Objective('val_R2', direction="max"),
            num_initial_points=50,
            max_trials=max_trials,
            overwrite = True,
            directory='D:/SGYL/SM_results_data/Bayesian_Opt/Full_ResNet/',
            project_name=('Full_ResNet_'+str(year)))
tuner.search(X_train,y_train,epochs=100,batch_size = 4096,validation_data=(X_validate,y_validate),shuffle = True,callbacks = [reduce_lr,callback_early_stopping],verbose = 2)

Trial 50 Complete [00h 02m 55s]
val_R2: 0.5404419302940369

Best val_R2 So Far: 0.9164506793022156
Total elapsed time: 02h 37m 04s
INFO:tensorflow:Oracle triggered exit


In [86]:
filepath="Full_ResNet_"+str(year)+"_best.hdf5"
callback_checkpoints = keras.callbacks.ModelCheckpoint(os.path.join('D:/SGYL/SM_results_data/check_points/Full_ResNet/',filepath),monitor='val_loss',save_best_only=True,verbose=1)
callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=50,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
callbacks = [reduce_lr,callback_early_stopping,callback_checkpoints]

if training_mode == 'model':
    model_best = tuner.get_best_models()[0]
else :
    model_best = tuner.hypermodel.build(tuner.get_best_hyperparameters()[0])
model_best.fit(X_train,y_train,epochs=300,batch_size = 4096,validation_data=(X_validate,y_validate),shuffle = True,callbacks = callbacks,verbose = 2)

Epoch 1/300

Epoch 00001: val_loss improved from inf to 0.00070, saving model to D:/SGYL/SM_results_data/check_points/Full_ResNet\Full_ResNet_2019_best.hdf5
153/153 - 2s - loss: 6.3179e-04 - R2: 0.9245 - mae: 0.0186 - val_loss: 6.9727e-04 - val_R2: 0.9162 - val_mae: 0.0195
Epoch 2/300

Epoch 00002: val_loss did not improve from 0.00070
153/153 - 2s - loss: 6.2934e-04 - R2: 0.9247 - mae: 0.0186 - val_loss: 6.9809e-04 - val_R2: 0.9161 - val_mae: 0.0194
Epoch 3/300

Epoch 00003: val_loss improved from 0.00070 to 0.00070, saving model to D:/SGYL/SM_results_data/check_points/Full_ResNet\Full_ResNet_2019_best.hdf5
153/153 - 2s - loss: 6.2889e-04 - R2: 0.9248 - mae: 0.0186 - val_loss: 6.9708e-04 - val_R2: 0.9162 - val_mae: 0.0194
Epoch 4/300

Epoch 00004: val_loss improved from 0.00070 to 0.00070, saving model to D:/SGYL/SM_results_data/check_points/Full_ResNet\Full_ResNet_2019_best.hdf5
153/153 - 2s - loss: 6.3018e-04 - R2: 0.9247 - mae: 0.0186 - val_loss: 6.9614e-04 - val_R2: 0.9163 - val_m

In [87]:
#save model
save_path = os.path.join('D:/SGYL/SM_results_data/model/Full_ResNet/','Full_ResNet_'+str(year)+'.hdf5')
model_best.save(save_path)
if (os.path.exists(save_path)):
    print('save model for year:',year)

#save model config
import json
bestConfig=tuner.get_best_hyperparameters()[0].get_config()
f = open(os.path.join('D:/SGYL/SM_results_data/Bayesian_Opt/Full_ResNet/Best_Config/','Full_ResNet_'+str(year)+'.json'), 'w')
json.dump(bestConfig,f)
f.close()
print(bestConfig['values'])

model_best.evaluate(X_test,y_test)

save model for year: 2019
{'Nodes_choice': 3, 'activation': 'tanh', 'activate_dense_last': 'sigmoid', 'lr': 0.0028838160759856015}
2514/2514 [==============================] - 6s 2ms/step - loss: 6.9373e-04 - R2: 0.9105 - mae: 0.0194


[0.0006937275757081807, 0.9104613065719604, 0.01938627101480961]

### 2020

In [88]:
from sklearn.preprocessing import StandardScaler
year = 2020
data_train = pd.read_csv(os.path.join('D:/SGYL/SM_results_data/Train/split/','train_data_'+str(year)+'.csv'))
data_validate = pd.read_csv(os.path.join('D:/SGYL/SM_results_data/Train/split/','validate_data_'+str(year)+'.csv'))
data_test = pd.read_csv(os.path.join('D:/SGYL/SM_results_data/Train/split/','test_data_'+str(year)+'.csv'))

X_train = data_train.drop(['SM'],axis = 1)
y_train = data_train['SM'].copy()   
X_validate = data_validate.drop(['SM'],axis = 1)
y_validate = data_validate['SM'].copy()
X_test = data_test.drop(['SM'],axis = 1)
y_test = data_test['SM'].copy()

standarder = StandardScaler()
X_train = standarder.fit_transform(X_train)
X_validate = standarder.transform(X_validate)
X_test = standarder.transform(X_test)

X_train = X_train.reshape(X_train.shape[0],X_train.shape[1],1)
X_validate = X_validate.reshape(X_validate.shape[0],X_validate.shape[1],1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1],1)

print('X_train shape:',X_train.shape)
print('X_validate shape:',X_validate.shape)

X_train shape: (709183, 19, 1)
X_validate shape: (212755, 19, 1)


In [89]:

def scheduler(epoch,lr):
    # 每隔30个epoch，学习率减小为原来的1/10
    if epoch % 30 == 0 and epoch != 0:
        print("lr changed to {}".format(lr * 0.1))
        return lr * 0.1
    else :
        return lr

callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=20,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
hypermodel = MyHyperModel()
tuner = keras_tuner.BayesianOptimization(
            hypermodel,
            objective=keras_tuner.Objective('val_R2', direction="max"),
            num_initial_points=50,
            max_trials=max_trials,
            overwrite = True,
            directory='D:/SGYL/SM_results_data/Bayesian_Opt/Full_ResNet/',
            project_name=('Full_ResNet_'+str(year)))
tuner.search(X_train,y_train,epochs=100,batch_size = 4096,validation_data=(X_validate,y_validate),shuffle = True,callbacks = [reduce_lr,callback_early_stopping],verbose = 2)

Trial 50 Complete [00h 03m 36s]
val_R2: 0.884468138217926

Best val_R2 So Far: 0.9061598181724548
Total elapsed time: 02h 52m 07s
INFO:tensorflow:Oracle triggered exit


In [90]:
filepath="Full_ResNet_"+str(year)+"_best.hdf5"
callback_checkpoints = keras.callbacks.ModelCheckpoint(os.path.join('D:/SGYL/SM_results_data/check_points/Full_ResNet/',filepath),monitor='val_loss',save_best_only=True,verbose=1)
callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=50,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
callbacks = [reduce_lr,callback_early_stopping,callback_checkpoints]

if training_mode == 'model':
    model_best = tuner.get_best_models()[0]
else :
    model_best = tuner.hypermodel.build(tuner.get_best_hyperparameters()[0])
model_best.fit(X_train,y_train,epochs=300,batch_size = 4096,validation_data=(X_validate,y_validate),shuffle = True,callbacks = callbacks,verbose = 2)

Epoch 1/300

Epoch 00001: val_loss improved from inf to 0.00073, saving model to D:/SGYL/SM_results_data/check_points/Full_ResNet\Full_ResNet_2020_best.hdf5
174/174 - 3s - loss: 7.0490e-04 - R2: 0.9094 - mae: 0.0199 - val_loss: 7.3005e-04 - val_R2: 0.9062 - val_mae: 0.0202
Epoch 2/300

Epoch 00002: val_loss did not improve from 0.00073
174/174 - 2s - loss: 7.0377e-04 - R2: 0.9095 - mae: 0.0199 - val_loss: 7.3138e-04 - val_R2: 0.9060 - val_mae: 0.0202
Epoch 3/300

Epoch 00003: val_loss did not improve from 0.00073
174/174 - 2s - loss: 7.0218e-04 - R2: 0.9097 - mae: 0.0198 - val_loss: 7.3237e-04 - val_R2: 0.9059 - val_mae: 0.0202
Epoch 4/300

Epoch 00004: val_loss did not improve from 0.00073
174/174 - 2s - loss: 7.0147e-04 - R2: 0.9097 - mae: 0.0198 - val_loss: 7.3006e-04 - val_R2: 0.9062 - val_mae: 0.0202
Epoch 5/300

Epoch 00005: val_loss did not improve from 0.00073
174/174 - 2s - loss: 7.0361e-04 - R2: 0.9096 - mae: 0.0199 - val_loss: 7.3344e-04 - val_R2: 0.9058 - val_mae: 0.0203
Ep

In [91]:
#save model
save_path = os.path.join('D:/SGYL/SM_results_data/model/Full_ResNet/','Full_ResNet_'+str(year)+'.hdf5')
model_best.save(save_path)
if (os.path.exists(save_path)):
    print('save model for year:',year)

#save model config
import json
bestConfig=tuner.get_best_hyperparameters()[0].get_config()
f = open(os.path.join('D:/SGYL/SM_results_data/Bayesian_Opt/Full_ResNet/Best_Config/','Full_ResNet_'+str(year)+'.json'), 'w')
json.dump(bestConfig,f)
f.close()
print(bestConfig['values'])

model_best.evaluate(X_test,y_test)

save model for year: 2020
{'Nodes_choice': 5, 'activation': 'tanh', 'activate_dense_last': 'sigmoid', 'lr': 0.0077584451667304585}
2850/2850 [==============================] - 8s 3ms/step - loss: 7.4874e-04 - R2: 0.8966 - mae: 0.0203


[0.0007487407419830561, 0.896561324596405, 0.020262479782104492]

### aaa